In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib as plt
from sklearn import preprocessing

import plotly.express as px

In [2]:
df = pd.read_excel('Spend & Orders by Channel.xlsx', sheet_name = 'Spent and Orders')
df.head()

,Date,Affiliate,All Other Non-Paid,All Other Paid,Direct,Display,Email,Organic Search,Organic Social,Paid Search,...,Affiliate_Spent,All Other Paid_Spent,Display_Spent,Email_Spent,Paid Social_Spent,Search Brand_Spent,Search NonBrand_Spent,Shopping Ads_Spent,TV_Spent,CTV_Spent
0,2018-12-29,93,20,0,284,226,263,195,5,561,...,1932.0,0.0,1709.0,18540.0,73526.0,7352.0,1505.0,6447.0,NaN,NaN
1,2018-12-30,84,23,0,288,202,264,198,10,558,...,2992.0,0.0,1393.0,20530.0,78580.0,8338.0,1412.0,6571.0,NaN,NaN
2,2018-12-31,79,17,0,232,164,297,149,8,401,...,4417.0,0.0,1704.0,21805.0,70136.0,5496.0,1221.0,5394.0,44579.95,5281.98
3,2019-01-01,107,14,0,384,188,56,230,4,660,...,2381.0,0.0,2184.0,2490.0,64916.0,9066.0,1599.0,8075.0,135627.40,5281.98
4,2019-01-02,117,35,0,386,189,303,278,4,715,...,6809.0,0.0,2144.0,22705.0,62960.0,9629.0,1461.0,6655.0,206023.73,5281.98


In [3]:
# Replace Column header spaces with '_'

df.columns = df.columns.str.replace(' ','_')

In [4]:
df['Total_Orders'] = df.iloc[:,1:18].sum(axis=1)

In [5]:
df['Total_Spent'] = df.iloc[:,18:28].sum(axis=1)

In [6]:
# Create Weekly Data:
df['Date_W'] = pd.to_datetime(df['Date']) - pd.to_timedelta(-7, unit='d')

In [7]:
df1 = df[(df['Date']>='2019-03-25') & (df['Date'] <='2019-09-29')]
df1.head()

,Date,Affiliate,All_Other_Non-Paid,All_Other_Paid,Direct,Display,Email,Organic_Search,Organic_Social,Paid_Search,...,Email_Spent,Paid_Social_Spent,Search_Brand_Spent,Search_NonBrand_Spent,Shopping_Ads_Spent,TV_Spent,CTV_Spent,Total_Orders,Total_Spent,Date_W
86,2019-03-25,160,21,0,411,2,161,370,40,652,...,9465.0,98090.0,9921.0,8223.0,8125.0,126961.272857,28647.400518,2290,298130.560075,2019-04-01
87,2019-03-26,92,26,0,404,4,120,334,28,542,...,7125.0,97364.0,9690.0,7209.0,8111.0,95128.332857,9793.960224,1970,241207.288381,2019-04-02
88,2019-03-27,83,26,0,371,5,89,372,22,543,...,5675.0,89398.0,9553.0,7046.0,8164.0,122272.392857,11662.707965,1920,262629.845022,2019-04-03
89,2019-03-28,82,18,0,404,5,91,321,18,548,...,5690.0,79294.0,9377.0,6994.0,7313.0,122115.222857,13606.175035,1921,251784.189092,2019-04-04
90,2019-03-29,92,50,0,422,4,92,354,32,540,...,5045.0,92606.0,9120.0,6632.0,7313.0,121276.122857,18387.691118,2040,272480.709175,2019-04-05


In [8]:
# Weekly Data for the entire dataset:

df_Weekly = df.groupby([pd.Grouper(key='Date_W', 
                 freq='W-MON')])[['Total_Orders',
                                'TV_Spent',
                                'CTV_Spent', 
                                 'Affiliate_Spent',
                                 'All_Other_Paid_Spent',
                                 'Display_Spent',
                                  'Email_Spent',
                                  'Paid_Social_Spent',
                                  'Search_Brand_Spent',
                                  'Search_NonBrand_Spent',
                                  'Shopping_Ads_Spent',
                                  'Affiliate',
                                  'All_Other_Non-Paid',
                                  'All_Other_Paid',
                                  'Direct',
                                  'Display',
                                  'Email',
                                  'Organic_Search',
                                  'Organic_Social',
                                  'Search_Brand_',
                                  'Search_Non-Brand',
                                  'Shopping_Ads',
                                  'TV', 'PR', 'Video', 'Partnership_Marketing', 'Paid_Social', 'Paid_Search'
                                 ]].sum().reset_index().sort_values('Date_W')
df_Weekly

,Date_W,Total_Orders,TV_Spent,CTV_Spent,Affiliate_Spent,All_Other_Paid_Spent,Display_Spent,Email_Spent,Paid_Social_Spent,Search_Brand_Spent,...,Organic_Social,Search_Brand_,Search_Non-Brand,Shopping_Ads,TV,PR,Video,Partnership_Marketing,Paid_Social,Paid_Search
0,2019-01-07,5139,4.457995e+04,5281.980000,9341.0000,0.0,4806.000000,60875.0,222242.000000,21186.000000,...,23,0,0,489,28,0,0,1,0,1520
1,2019-01-14,17528,1.423501e+06,53282.684391,32487.0000,0.0,22603.867800,113250.0,863655.280000,74144.000000,...,112,0,0,1401,115,0,0,0,0,5554
2,2019-01-21,15620,1.253392e+06,81988.953374,25538.0200,0.0,31887.380200,58130.0,913537.380000,76371.000000,...,89,0,0,1410,127,0,0,2,0,4118
3,2019-01-28,15447,1.163752e+06,97872.503779,31314.3525,0.0,24891.773000,52440.0,853066.000000,53328.000000,...,162,0,0,1345,181,0,0,2,0,4665
4,2019-02-04,14482,1.197811e+06,79466.819802,21984.0000,0.0,17343.175300,47120.0,758180.000000,53574.000000,...,202,0,0,1370,158,0,0,0,0,4418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,2020-03-30,7704,5.796539e+05,49591.452361,10908.0300,0.0,1691.863027,4344.0,176026.659864,69694.640119,...,188,2877,111,0,33,0,0,0,255,0
65,2020-04-06,8906,5.066002e+05,42535.463195,13223.0600,0.0,2851.203276,4344.0,193718.720534,62151.490068,...,285,3233,41,0,20,0,0,0,357,0
66,2020-04-13,10304,4.619123e+05,50137.407581,6754.6000,0.0,6112.032658,5244.0,186192.199860,76799.949891,...,365,4058,34,0,20,0,0,0,400,0
67,2020-04-20,10231,4.713927e+05,51462.522297,5049.3400,0.0,3209.149323,5544.0,215873.420274,75980.010488,...,383,4102,58,0,20,0,0,0,528,0


In [12]:
# Weekly Data for the selected Date Range:

df1_Weekly = df1.groupby([ pd.Grouper(key='Date_W', 
                 freq='W-MON')])[['Total_Orders','Total_Spent',
                                'TV_Spent',
                                'CTV_Spent', 
                                 'Affiliate_Spent',
                                 'All_Other_Paid_Spent',
                                 'Display_Spent',
                                  'Email_Spent',
                                  'Paid_Social_Spent',
                                  'Search_Brand_Spent',
                                  'Search_NonBrand_Spent',
                                  'Shopping_Ads_Spent',
                                  'Affiliate',
                                  'All_Other_Non-Paid',
                                  'All_Other_Paid',
                                  'Direct',
                                  'Display',
                                  'Email',
                                  'Organic_Search',
                                  'Organic_Social',
                                  'Search_Brand_',
                                  'Search_Non-Brand',
                                  'Shopping_Ads',
                                  'TV', 'PR', 'Video', 'Partnership_Marketing', 'Paid_Social', 'Paid_Search'
                                 ]].sum().reset_index().sort_values('Date_W')
df1_Weekly

,Date_W,Total_Orders,Total_Spent,TV_Spent,CTV_Spent,Affiliate_Spent,All_Other_Paid_Spent,Display_Spent,Email_Spent,Paid_Social_Spent,...,Organic_Social,Search_Brand_,Search_Non-Brand,Shopping_Ads,TV,PR,Video,Partnership_Marketing,Paid_Social,Paid_Search
0,2019-04-01,2290,2.981306e+05,126961.272857,28647.400518,3452.0,682.00,4563.8867,9465.0,98090.0,...,40,0,0,198,55,0,0,0,220,652
1,2019-04-08,13870,1.699791e+06,698396.811429,95226.569098,27914.0,2889.00,25369.5524,43590.0,639228.0,...,184,0,0,1300,198,0,0,0,1559,3837
2,2019-04-15,13890,1.847422e+06,741327.837143,204349.057452,22417.0,3232.00,27486.9205,42525.0,657272.0,...,228,0,0,1303,86,1,1,0,1711,3586
3,2019-04-22,11304,1.670037e+06,733159.312857,197111.200361,20419.0,3130.00,19863.0551,38355.0,509110.0,...,165,0,0,968,75,0,0,1,1198,3978
4,2019-04-29,10941,1.502157e+06,695263.585714,235296.214902,18142.0,2082.00,19251.7472,36270.0,366146.0,...,211,0,0,893,59,0,0,0,935,4029
5,2019-05-06,10631,1.567191e+06,764150.977143,173781.860845,18661.0,27420.00,17642.0664,30575.0,407940.0,...,143,0,0,901,79,0,0,0,1096,3586
6,2019-05-13,10592,1.376344e+06,532366.115714,177790.641328,22079.0,48226.00,17936.2732,29172.0,431104.0,...,121,0,0,953,59,0,0,0,1079,3573
7,2019-05-20,9777,1.302502e+06,420398.032857,252135.366172,20479.0,32338.00,17907.0000,38654.0,417494.0,...,107,0,0,802,44,0,0,2,967,3295
8,2019-05-27,10586,1.423689e+06,417944.885714,208952.506986,25699.0,20961.00,17649.0000,29964.0,579384.0,...,123,0,0,885,62,1,1,0,1035,3311
9,2019-06-03,9043,1.357086e+06,488219.011429,171538.509227,27812.0,5047.00,14779.0000,25754.0,508060.0,...,153,0,0,845,45,0,0,0,679,2790


In [13]:
df1_Weekly['nonTV_Spent'] = df1_Weekly.iloc[:, 4:12].sum(axis=1)
df1_Weekly['Social_Orders'] = df1_Weekly['Organic_Social'] + df1_Weekly['Paid_Social']

In [14]:
df1_Weekly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date_W                 28 non-null     datetime64[ns]
 1   Total_Orders           28 non-null     int64         
 2   Total_Spent            28 non-null     float64       
 3   TV_Spent               28 non-null     float64       
 4   CTV_Spent              28 non-null     float64       
 5   Affiliate_Spent        28 non-null     float64       
 6   All_Other_Paid_Spent   28 non-null     float64       
 7   Display_Spent          28 non-null     float64       
 8   Email_Spent            28 non-null     float64       
 9   Paid_Social_Spent      28 non-null     float64       
 10  Search_Brand_Spent     28 non-null     float64       
 11  Search_NonBrand_Spent  28 non-null     float64       
 12  Shopping_Ads_Spent     28 non-null     float64       
 13  Affilia

# Total Orders Attribution

In [120]:
y= df1_Weekly["Total_Orders"]
X= df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Total_Orders   R-squared (uncentered):                   0.991
Model:                            OLS   Adj. R-squared (uncentered):              0.990
Method:                 Least Squares   F-statistic:                              907.1
Date:                Wed, 22 Jul 2020   Prob (F-statistic):                    1.26e-25
Time:                        15:36:21   Log-Likelihood:                         -228.67
No. Observations:                  28   AIC:                                      463.3
Df Residuals:                      25   BIC:                                      467.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0084      0.002      5.148      0.000       0.005       0.012
CTV_Spent       0.0117      0.004      3.223      0.004       0.004       0.019
nonTV_Spent     0.0060      0.001      4.319      0.000       0.003       0.009
==============================================================================
Omnibus:                        0.254   Durbin-Watson:                   0.793
Prob(Omnibus):                  0.881   Jarque-Bera (JB):                0.314
Skew:                          -0.198   Prob(JB):                        0.855
Kurtosis:                       2.666   Cond. No.                         15.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [121]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                 Coef
TV_Spent     0.008355
CTV_Spent    0.011700
nonTV_Spent  0.005967


In [122]:
# Total Predicted Orders by the Model:
((df1_Weekly['TV_Spent']* TV_Coef) + 
(df1_Weekly['CTV_Spent']* CTV_Coef)+ 
(df1_Weekly['nonTV_Spent']* nonTV_Coef)).sum()

236782.14950097102

In [123]:
# Actual Total Orders:
df1_Weekly["Total_Orders"].sum()

241102

In [124]:
# TV Attributed Orders Based on Total Orders:
print(f"TV Attributed Orders: {round((df1_Weekly['TV_Spent']* TV_Coef).sum(),0)} \n")
print(f"TV Attributed Orders: {round(df1_Weekly['TV_Spent'].sum() /(df1_Weekly['TV_Spent']* TV_Coef).sum(),2)} \n")

TV Attributed Orders: 101229.0 

TV Attributed Orders: 119.69 



In [125]:
# CTV Attributed Orders Based on Total Orders:
print(f"CTV Attributed Orders: {round((df1_Weekly['CTV_Spent']* CTV_Coef).sum(),0)}\n")
print(f"CTV CPO: {round((df1_Weekly['CTV_Spent'].sum()/(df1_Weekly['CTV_Spent']* CTV_Coef).sum()),2)}\n")

CTV Attributed Orders: 50291.0

CTV CPO: 85.47



In [126]:
df1_Weekly['Total_Orders_Res'] = df1_Weekly["Total_Orders"] - round((df1_Weekly['TV_Spent']* TV_Coef),0) - round((df1_Weekly['CTV_Spent']* CTV_Coef),0)


In [127]:
y = df1_Weekly["Total_Orders_Res"]
X = df1_Weekly.iloc[:,[4,10]]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:       Total_Orders_Res   R-squared (uncentered):                   0.965
Model:                            OLS   Adj. R-squared (uncentered):              0.963
Method:                 Least Squares   F-statistic:                              361.9
Date:                Wed, 22 Jul 2020   Prob (F-statistic):                    1.05e-19
Time:                        15:38:28   Log-Likelihood:                         -220.27
No. Observations:                  28   AIC:                                      444.5
Df Residuals:                      26   BIC:                                      447.2
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent           0.1056      0.007     16.129      0.000       0.092       0.119
Search_NonBrand_Spent     0.0524      0.008      6.211      0.000       0.035       0.070
==============================================================================
Omnibus:                        1.340   Durbin-Watson:                   1.698
Prob(Omnibus):                  0.512   Jarque-Bera (JB):                0.433
Skew:                          -0.231   Prob(JB):                        0.805
Kurtosis:                       3.397   Cond. No.                         2.24
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Paid Search Attribution

In [11]:
y= df1_Weekly["Paid_Search"]
X= df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:            Paid_Search   R-squared (uncentered):                   0.986
Model:                            OLS   Adj. R-squared (uncentered):              0.984
Method:                 Least Squares   F-statistic:                              569.8
Date:                Wed, 22 Jul 2020   Prob (F-statistic):                    3.94e-23
Time:                        11:43:16   Log-Likelihood:                         -204.23
No. Observations:                  28   AIC:                                      414.5
Df Residuals:                      25   BIC:                                      418.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0033      0.001      4.927      0.000       0.002       0.005
CTV_Spent       0.0078      0.002      5.150      0.000       0.005       0.011
nonTV_Spent     0.0003      0.001      0.603      0.552      -0.001       0.002
==============================================================================
Omnibus:                        1.126   Durbin-Watson:                   1.254
Prob(Omnibus):                  0.570   Jarque-Bera (JB):                0.730
Skew:                          -0.393   Prob(JB):                        0.694
Kurtosis:                       2.906   Cond. No.                         15.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                 Coef
TV_Spent     0.003341
CTV_Spent    0.007812
nonTV_Spent  0.000348


In [13]:
df1_Weekly["Paid_Search"].sum()

80914

In [14]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Paid_Search_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + 
                                  (df1_Weekly['CTV_Spent']* CTV_Coef)+ 
                                  (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['Paid_Search_Pred'].sum()

79027.8901410132

In [15]:
# Create the two new columns for TV and CTV Attributed Orders based on Paid Search:

df1_Weekly['TV_Att_PaidSearch'] = (df1_Weekly['TV_Spent']* TV_Coef)
df1_Weekly['CTV_Att_PaidSearch'] = (df1_Weekly['CTV_Spent']* CTV_Coef)

In [16]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['TV_Att_PaidSearch'] = ((df1_Weekly['TV_Att_PaidSearch']/df1_Weekly['Paid_Search_Pred'])*df1_Weekly['Paid_Search'])
df1_Weekly['CTV_Att_PaidSearch'] = ((df1_Weekly['CTV_Att_PaidSearch']/df1_Weekly['Paid_Search_Pred'])*df1_Weekly['Paid_Search'])


In [20]:
(df1_Weekly[['TV_Att_PaidSearch','CTV_Att_PaidSearch']]).sum().sum()

75830.7636501959

In [21]:
 # Calculate the residuals:
df1_Weekly["Paid_Search_Res"] = df1_Weekly['Paid_Search'] - df1_Weekly['TV_Att_PaidSearch'] - df1_Weekly['CTV_Att_PaidSearch']

In [27]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Paid_Search_Res"]
X1= df1_Weekly[['Affiliate_Spent', 
                'All_Other_Paid_Spent', 
                'Display_Spent',
               'Email_Spent',
               'Paid_Social_Spent',
                'Search_Brand_Spent',
               'Search_NonBrand_Spent',
               'Shopping_Ads_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:        Paid_Search_Res   R-squared (uncentered):                   0.994
Model:                            OLS   Adj. R-squared (uncentered):              0.991
Method:                 Least Squares   F-statistic:                              402.3
Date:                Wed, 22 Jul 2020   Prob (F-statistic):                    2.27e-20
Time:                        11:53:14   Log-Likelihood:                         -115.90
No. Observations:                  28   AIC:                                      247.8
Df Residuals:                      20   BIC:                                      258.5
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent           0.0029      0.001      2.838      0.010       0.001       0.005
All_Other_Paid_Spent      0.0008      0.000      2.143      0.045     2.1e-05       0.002
Display_Spent             0.0003      0.002      0.198      0.845      -0.003       0.004
Email_Spent              -0.0016      0.001     -1.235      0.231      -0.004       0.001
Paid_Social_Spent         0.0001   8.43e-05      1.495      0.151   -4.98e-05       0.000
Search_Brand_Spent        0.0003      0.001      0.488      0.631      -0.001       0.001
Search_NonBrand_Spent     0.0028      0.001      2.562      0.019       0.001       0.005
Shopping_Ads_Spent        0.0012      0.001      1.335      0.197      -0.001       0.003
==============================================================================
Omnibus:                       11.696   Durbin-Watson:                   2.491
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               16.363
Skew:                          -0.810   Prob(JB):                     0.000280
Kurtosis:                       6.376   Cond. No.                         202.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Paid_Search_Res"]
X1= df1_Weekly[['Affiliate_Spent', 
                'All_Other_Paid_Spent', 
               'Paid_Social_Spent',
               'Search_NonBrand_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:        Paid_Search_Res   R-squared (uncentered):                   0.993
Model:                            OLS   Adj. R-squared (uncentered):              0.992
Method:                 Least Squares   F-statistic:                              849.2
Date:                Wed, 22 Jul 2020   Prob (F-statistic):                    1.84e-25
Time:                        11:55:11   Log-Likelihood:                         -117.69
No. Observations:                  28   AIC:                                      243.4
Df Residuals:                      24   BIC:                                      248.7
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent           0.0035      0.001      5.032      0.000       0.002       0.005
All_Other_Paid_Spent      0.0008      0.000      2.417      0.024       0.000       0.001
Paid_Social_Spent         0.0002   6.98e-05      2.309      0.030    1.71e-05       0.000
Search_NonBrand_Spent     0.0025      0.001      3.687      0.001       0.001       0.004
==============================================================================
Omnibus:                        2.560   Durbin-Watson:                   2.563
Prob(Omnibus):                  0.278   Jarque-Bera (JB):                1.263
Skew:                          -0.190   Prob(JB):                        0.532
Kurtosis:                       3.968   Cond. No.                         112.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
# Get the Model Coefficients:
Affiliate_Coef = result.params[0]
All_Other_Paid_Coef = result.params[1]
Paid_Social_Coef = result.params[2]
Search_NonBrand_Coef = result.params[3]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                           Coef
Affiliate_Spent        0.003494
All_Other_Paid_Spent   0.000762
Paid_Social_Spent      0.000161
Search_NonBrand_Spent  0.002466


In [66]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Paid_Search_Res_Pred'] = ((df1_Weekly['Affiliate_Spent']* Affiliate_Coef) + 
                                  (df1_Weekly['All_Other_Paid_Spent']* All_Other_Paid_Coef)+ 
                                  (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef) + 
                                  (df1_Weekly['Search_NonBrand_Spent']* Search_NonBrand_Coef))

df1_Weekly['Paid_Search_Res_Pred'].sum()

5089.321186769879

In [83]:
# Create the two new columns for Attributed Orders based on Paid Search:

df1_Weekly['Affiliate_Att_PaidSearch'] = (df1_Weekly['Affiliate_Spent']* Affiliate_Coef) 
df1_Weekly['All_Other_Paid_Att_PaidSearch'] = (df1_Weekly['All_Other_Paid_Spent']* All_Other_Paid_Coef)
df1_Weekly['Paid_Social_Att_PaidSearch'] = (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)
df1_Weekly['Search_NonBrand_Att_PaidSearch'] = (df1_Weekly['Search_NonBrand_Spent']* Search_NonBrand_Coef)


In [85]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['Affiliate_Att_PaidSearch'] = ((df1_Weekly['Affiliate_Att_PaidSearch']/df1_Weekly['Paid_Search_Res_Pred'])*df1_Weekly['Paid_Search_Res'])
df1_Weekly['All_Other_Paid_Att_PaidSearch'] = ((df1_Weekly['All_Other_Paid_Att_PaidSearch']/df1_Weekly['Paid_Search_Res_Pred'])*df1_Weekly['Paid_Search_Res'])

df1_Weekly['Paid_Social_Att_PaidSearch'] = ((df1_Weekly['Paid_Social_Att_PaidSearch']/df1_Weekly['Paid_Search_Res_Pred'])*df1_Weekly['Paid_Search_Res'])
df1_Weekly['Search_NonBrand_Att_PaidSearch'] = ((df1_Weekly['Search_NonBrand_Att_PaidSearch']/df1_Weekly['Paid_Search_Res_Pred'])*df1_Weekly['Paid_Search_Res'])


In [86]:
print(f" Paid_Search_Res_Pred: {df1_Weekly['Paid_Search_Res_Pred'].sum()} \n \
Paid_Search_Res: {df1_Weekly['Paid_Search_Res'].sum()} \n \
{df1_Weekly[['Affiliate_Att_PaidSearch','All_Other_Paid_Att_PaidSearch','Paid_Social_Att_PaidSearch','Search_NonBrand_Att_PaidSearch']].sum()} \
" )

 Paid_Search_Res_Pred: 5089.321186769879 
 Paid_Search_Res: 5083.23634980408 
 Affiliate_Att_PaidSearch          2350.562813
All_Other_Paid_Att_PaidSearch      220.094497
Paid_Social_Att_PaidSearch        1641.305771
Search_NonBrand_Att_PaidSearch     871.273269
dtype: float64 


In [95]:
# Sanity Check to Make sure the two residual Number Match:

print(f" Residuals Attributed Total Orders: {df1_Weekly[['Affiliate_Att_PaidSearch','All_Other_Paid_Att_PaidSearch','Paid_Social_Att_PaidSearch','Search_NonBrand_Att_PaidSearch']].sum().sum()}")

print(f" Residuals Total Orders: {df1_Weekly['Paid_Search_Res'].sum()}")

 Residuals Attributed Total Orders: 5083.2363498040795
 Residuals Total Orders: 5083.23634980408


In [110]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_Paid_Search = df1_Weekly[['Date_W', 'TV_Att_PaidSearch','CTV_Att_PaidSearch',
                             'Affiliate_Att_PaidSearch', 'All_Other_Paid_Att_PaidSearch', 
                             'Paid_Social_Att_PaidSearch', 'Search_NonBrand_Att_PaidSearch']].rename(columns= {'TV_Att_PaidSearch': 'TV_attributed',
                                'CTV_Att_PaidSearch':'CTV_attributed',
                                'Affiliate_Att_PaidSearch': 'Affiliate_attributed', 
                                'All_Other_Paid_Att_PaidSearch': 'All_Other_Paid_attributed', 
                                'Paid_Social_Att_PaidSearch' :'Paid_Social_attributed', 
                                'Search_NonBrand_Att_PaidSearch': 'Search_NonBrand_attributed' })
df_Paid_Search

,Date_W,TV_attributed,CTV_attributed,Affiliate_attributed,All_Other_Paid_attributed,Paid_Social_attributed,Search_NonBrand_attributed
0,2019-04-01,396.433430,209.169878,11.499916,0.495247,15.069840,19.331689
1,2019-04-08,2638.723780,841.326836,107.677110,2.429194,113.714839,133.128242
2,2019-04-15,2024.389407,1304.882202,72.606463,2.281825,98.175462,83.664641
3,2019-04-22,2294.284069,1442.364610,69.608899,2.325883,80.039057,89.377482
4,2019-04-29,2146.413814,1698.610612,64.346677,1.609663,59.890134,58.129100
5,2019-05-06,2216.805360,1178.874991,60.806654,19.475873,61.301700,48.735423
6,2019-05-13,1869.313531,1459.807403,79.008559,37.617443,71.143762,56.109302
7,2019-05-20,1287.772272,1806.038826,67.027957,23.071385,63.016968,48.072592
8,2019-05-27,1398.321086,1634.746434,93.020678,16.538212,96.714104,71.659486
9,2019-06-03,1415.873925,1163.285861,79.634359,3.150031,67.087763,60.968061


In [111]:
df_Paid_Search['Channel'] = 'Paid_Search'

#df_Paid_Search['Affiliate_attributed'] = 0
#df_Paid_Search['All_Other_Paid_attributed'] = 0
df_Paid_Search['Display_attributed'] = 0
df_Paid_Search['Email_attributed'] = 0
df_Paid_Search['SearchBrand_attributed'] = 0
#df_Paid_Search['Search_NonBrand_attributed'] = 0
df_Paid_Search['ShoppingAds_attributed'] = 0
#df_Paid_Search['Paid_Social_attributed'] = 0 
#df_Paid_Search['TV_attributed'] = 0 
#df_Paid_Search['CTV_attributed'] = 0 

In [113]:
# Paid Search Attributed Orders:

df_Paid_Search = df_Paid_Search[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]

In [143]:
df_Paid_Search.iloc[:,2:12].sum()#.sum()

TV_attributed                 41469.738341
CTV_attributed                34361.025310
Paid_Social_attributed         1641.305771
Affiliate_attributed           2350.562813
All_Other_Paid_attributed       220.094497
Display_attributed                0.000000
Email_attributed                  0.000000
SearchBrand_attributed            0.000000
Search_NonBrand_attributed      871.273269
ShoppingAds_attributed            0.000000
dtype: float64

In [170]:
df_Paid_Search.iloc[:,2:12].sum().sum()

80913.99999999999

# Paid Search & Organic Search Combined Attribution:


In [15]:
y = df1_Weekly[['Organic_Search', 'Paid_Search']].sum(axis=1)
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.987
Model:                            OLS   Adj. R-squared (uncentered):              0.985
Method:                 Least Squares   F-statistic:                              634.3
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    1.05e-23
Time:                        09:54:59   Log-Likelihood:                         -212.99
No. Observations:                  28   AIC:                                      432.0
Df Residuals:                      25   BIC:                                      436.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0052      0.001      5.640      0.000       0.003       0.007
CTV_Spent       0.0056      0.003      2.211      0.036       0.000       0.011
nonTV_Spent     0.0015      0.001      1.792      0.085      -0.000       0.003
==============================================================================
Omnibus:                        0.446   Durbin-Watson:                   0.912
Prob(Omnibus):                  0.800   Jarque-Bera (JB):                0.465
Skew:                          -0.264   Prob(JB):                        0.793
Kurtosis:                       2.655   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                 Coef
TV_Spent     0.005184
CTV_Spent    0.005607
nonTV_Spent  0.001499


In [17]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Paid_Organic_Search_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + 
                                  (df1_Weekly['CTV_Spent']* CTV_Coef)+ 
                                  (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['Paid_Organic_Search_Pred'].sum()

113475.73951915062

In [18]:
y.sum()

115933

In [19]:
# Create the two new columns for TV and CTV Attributed Orders based on Paid Search:

df1_Weekly['TV_Att_PaidOrganicSearch'] = (df1_Weekly['TV_Spent']* TV_Coef)
df1_Weekly['CTV_Att_PaidOrganicSearch'] = (df1_Weekly['CTV_Spent']* CTV_Coef)

In [20]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['TV_Att_PaidOrganicSearch'] = ((df1_Weekly['TV_Att_PaidOrganicSearch']/df1_Weekly['Paid_Organic_Search_Pred'])*y)
df1_Weekly['CTV_Att_PaidOrganicSearch'] = ((df1_Weekly['CTV_Att_PaidOrganicSearch']/df1_Weekly['Paid_Organic_Search_Pred'])*y)


In [21]:
 # Calculate the residuals:
df1_Weekly["PaidOrganic_Search_Res"] = y - df1_Weekly['TV_Att_PaidOrganicSearch'] - df1_Weekly['CTV_Att_PaidOrganicSearch']
df1_Weekly["PaidOrganic_Search_Res"].sum()

27078.040666554713

In [22]:
(df1_Weekly[['TV_Att_PaidOrganicSearch','CTV_Att_PaidOrganicSearch']]).sum()

TV_Att_PaidOrganicSearch     64209.144089
CTV_Att_PaidOrganicSearch    24645.815244
dtype: float64

In [23]:
df1_Weekly["PaidOrganic_Search_Res"].sum() + df1_Weekly['TV_Att_PaidOrganicSearch'].sum() + df1_Weekly['CTV_Att_PaidOrganicSearch'].sum()

115933.0

In [24]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["PaidOrganic_Search_Res"]
X1= df1_Weekly[['Affiliate_Spent', 
                'All_Other_Paid_Spent', 
                'Display_Spent',
               'Email_Spent',
               'Paid_Social_Spent',
                'Search_Brand_Spent',
               'Search_NonBrand_Spent',
               'Shopping_Ads_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     PaidOrganic_Search_Res   R-squared (uncentered):                   0.996
Model:                                OLS   Adj. R-squared (uncentered):              0.995
Method:                     Least Squares   F-statistic:                              641.7
Date:                    Thu, 23 Jul 2020   Prob (F-statistic):                    2.19e-22
Time:                            09:55:11   Log-Likelihood:                         -155.73
No. Observations:                      28   AIC:                                      327.5
Df Residuals:                          20   BIC:                                      338.1
Df Model:                               8                                                  
Covariance Type:                nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent           0.0118      0.004      2.791      0.011       0.003       0.021
All_Other_Paid_Spent      0.0025      0.002      1.617      0.121      -0.001       0.006
Display_Spent            -0.0033      0.007     -0.502      0.621      -0.017       0.010
Email_Spent               0.0096      0.005      1.804      0.086      -0.002       0.021
Paid_Social_Spent         0.0016      0.000      4.462      0.000       0.001       0.002
Search_Brand_Spent        0.0023      0.002      0.978      0.340      -0.003       0.007
Search_NonBrand_Spent    -0.0016      0.005     -0.354      0.727      -0.011       0.008
Shopping_Ads_Spent       -0.0040      0.004     -1.093      0.288      -0.012       0.004
==============================================================================
Omnibus:                        0.881   Durbin-Watson:                   2.020
Prob(Omnibus):                  0.644   Jarque-Bera (JB):                0.622
Skew:                          -0.355   Prob(JB):                        0.733
Kurtosis:                       2.832   Cond. No.                         202.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["PaidOrganic_Search_Res"]
X1= df1_Weekly[['Affiliate_Spent','Paid_Social_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     PaidOrganic_Search_Res   R-squared (uncentered):                   0.992
Model:                                OLS   Adj. R-squared (uncentered):              0.991
Method:                     Least Squares   F-statistic:                              1614.
Date:                    Thu, 23 Jul 2020   Prob (F-statistic):                    5.43e-28
Time:                            09:55:13   Log-Likelihood:                         -165.84
No. Observations:                      28   AIC:                                      335.7
Df Residuals:                          26   BIC:                                      338.4
Df Model:                               2                                                  
Covariance Type:                nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Affiliate_Spent       0.0107      0.002      5.549      0.000       0.007       0.015
Paid_Social_Spent     0.0019      0.000     16.061      0.000       0.002       0.002
==============================================================================
Omnibus:                        0.957   Durbin-Watson:                   1.175
Prob(Omnibus):                  0.620   Jarque-Bera (JB):                0.255
Skew:                          -0.196   Prob(JB):                        0.880
Kurtosis:                       3.254   Cond. No.                         42.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# Get the Model Coefficients:
Affiliate_Coef = result.params[0]
Paid_Social_Coef = result.params[1]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                       Coef
Affiliate_Spent    0.010650
Paid_Social_Spent  0.001936


In [27]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['PaidOrganic_Search_Res_Pred'] = ((df1_Weekly['Affiliate_Spent']* Affiliate_Coef) + 
                                  (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)) 
df1_Weekly['PaidOrganic_Search_Res_Pred'].sum()

26905.11828460486

In [28]:
# Create the two new columns for Attributed Orders based on Paid Search:

df1_Weekly['Affiliate_Att_PaidOrganic_Search'] = (df1_Weekly['Affiliate_Spent']* Affiliate_Coef) 

df1_Weekly['Paid_Social_Att_PaidOrganic_Search'] = (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)


In [29]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['Affiliate_Att_PaidOrganic_Search'] = ((df1_Weekly['Affiliate_Att_PaidOrganic_Search']/df1_Weekly['PaidOrganic_Search_Res_Pred'])*df1_Weekly['PaidOrganic_Search_Res'])

df1_Weekly['Paid_Social_Att_PaidOrganic_Search'] = ((df1_Weekly['Paid_Social_Att_PaidOrganic_Search']/df1_Weekly['PaidOrganic_Search_Res_Pred'])*df1_Weekly['PaidOrganic_Search_Res'])



In [30]:
print(f"PaidOrganic_Search_Res_Pred: {df1_Weekly['PaidOrganic_Search_Res_Pred'].sum()} \n\
PaidOrganic_Search_Res: {df1_Weekly['PaidOrganic_Search_Res'].sum()} \n\
{df1_Weekly[['Affiliate_Att_PaidOrganic_Search','Paid_Social_Att_PaidOrganic_Search']].sum()} \n\
Total_Residuals: {df1_Weekly[['Affiliate_Att_PaidOrganic_Search','Paid_Social_Att_PaidOrganic_Search']].sum().sum()}" )

PaidOrganic_Search_Res_Pred: 26905.11828460486 
PaidOrganic_Search_Res: 27078.040666554713 
Affiliate_Att_PaidOrganic_Search       7219.040504
Paid_Social_Att_PaidOrganic_Search    19859.000162
dtype: float64 
Total_Residuals: 27078.040666554705


In [31]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_PaidOrganic_Search = df1_Weekly[['Date_W', 'TV_Att_PaidOrganicSearch','CTV_Att_PaidOrganicSearch',
                             'Affiliate_Att_PaidOrganic_Search', 
                             'Paid_Social_Att_PaidOrganic_Search']].rename(columns= {'TV_Att_PaidOrganicSearch': 'TV_attributed',
                                'CTV_Att_PaidOrganicSearch':'CTV_attributed',
                                'Affiliate_Att_PaidOrganic_Search': 'Affiliate_attributed', 
                                'Paid_Social_Att_PaidOrganic_Search' :'Paid_Social_attributed'})
df_PaidOrganic_Search.head(2)

,Date_W,TV_attributed,CTV_attributed,Affiliate_attributed,Paid_Social_attributed
0,2019-04-01,632.651954,154.403518,38.110405,196.834123
1,2019-04-08,4161.802563,613.784264,316.410017,1317.003156


In [32]:
df_PaidOrganic_Search['Channel'] = 'Paid_Organic_Search'

#df_PaidOrganic_Search['Affiliate_attributed'] = 0
df_PaidOrganic_Search['All_Other_Paid_attributed'] = 0
df_PaidOrganic_Search['Display_attributed'] = 0
df_PaidOrganic_Search['Email_attributed'] = 0
df_PaidOrganic_Search['SearchBrand_attributed'] = 0
df_PaidOrganic_Search['Search_NonBrand_attributed'] = 0
df_PaidOrganic_Search['ShoppingAds_attributed'] = 0
#df_PaidOrganic_Search['Paid_Social_attributed'] = 0 
#df_PaidOrganic_Search['TV_attributed'] = 0 
#df_PaidOrganic_Search['CTV_attributed'] = 0 

In [33]:
# Paid Search Attributed Orders:

df_PaidOrganic_Search = df_PaidOrganic_Search[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_PaidOrganic_Search.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Paid_Organic_Search,632.651954,154.403518,196.834123,38.110405,0,0,0,0,0,0
1,2019-04-08,Paid_Organic_Search,4161.802563,613.784264,1317.003156,316.410017,0,0,0,0,0,0
2,2019-04-15,Paid_Organic_Search,3778.494482,1126.573444,1307.576184,245.355889,0,0,0,0,0,0
3,2019-04-22,Paid_Organic_Search,3364.830229,978.485718,968.890476,213.793577,0,0,0,0,0,0
4,2019-04-29,Paid_Organic_Search,3276.926758,1199.528521,818.437545,223.107175,0,0,0,0,0,0


In [34]:
df_PaidOrganic_Search.iloc[:,2:12].sum()

TV_attributed                 64209.144089
CTV_attributed                24645.815244
Paid_Social_attributed        19859.000162
Affiliate_attributed           7219.040504
All_Other_Paid_attributed         0.000000
Display_attributed                0.000000
Email_attributed                  0.000000
SearchBrand_attributed            0.000000
Search_NonBrand_attributed        0.000000
ShoppingAds_attributed            0.000000
dtype: float64

In [35]:
df_PaidOrganic_Search.iloc[:,2:12].sum().sum()

115932.99999999999

# Organic Search Attribution:


In [36]:
y = df1_Weekly[['Organic_Search']]
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:         Organic_Search   R-squared (uncentered):                   0.966
Model:                            OLS   Adj. R-squared (uncentered):              0.961
Method:                 Least Squares   F-statistic:                              233.5
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    2.10e-18
Time:                        09:55:34   Log-Likelihood:                         -194.12
No. Observations:                  28   AIC:                                      394.2
Df Residuals:                      25   BIC:                                      398.2
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0018      0.000      3.906      0.001       0.001       0.003
CTV_Spent      -0.0018      0.001     -1.430      0.165      -0.005       0.001
nonTV_Spent     0.0011      0.000      2.675      0.013       0.000       0.002
==============================================================================
Omnibus:                        1.466   Durbin-Watson:                   1.232
Prob(Omnibus):                  0.481   Jarque-Bera (JB):                0.472
Skew:                           0.201   Prob(JB):                        0.790
Kurtosis:                       3.493   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [178]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                 Coef
TV_Spent     0.001831
CTV_Spent   -0.000657
nonTV_Spent  0.001055


In [180]:
df1_Weekly['TV_Att_OrganicSearch'] = (df1_Weekly['TV_Spent']* TV_Coef) 

In [181]:
# Calculate the residuals:
df1_Weekly["Organic_Search_Res"] = df1_Weekly['Organic_Search'] - df1_Weekly['TV_Att_OrganicSearch']


In [185]:
#Sanity Check:

print(df1_Weekly["Organic_Search_Res"].sum())
print(df1_Weekly['TV_Att_OrganicSearch'].sum())
print(df1_Weekly['Organic_Search'].sum())
print(df1_Weekly['TV_Att_OrganicSearch'].sum() + df1_Weekly["Organic_Search_Res"].sum() )

12833.164433530192
22185.835566469803
35019
35018.99999999999


In [194]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Organic_Search_Res"]
X1= df1_Weekly[['Paid_Social_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Organic_Search_Res   R-squared (uncentered):                   0.793
Model:                            OLS   Adj. R-squared (uncentered):              0.785
Method:                 Least Squares   F-statistic:                              103.3
Date:                Wed, 22 Jul 2020   Prob (F-statistic):                    1.00e-10
Time:                        20:48:06   Log-Likelihood:                         -193.98
No. Observations:                  28   AIC:                                      390.0
Df Residuals:                      27   BIC:                                      391.3
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Paid_Social_Spent     0.0012      0.000     10.163      0.000       0.001       0.001
==============================================================================
Omnibus:                        3.960   Durbin-Watson:                   1.233
Prob(Omnibus):                  0.138   Jarque-Bera (JB):                2.331
Skew:                           0.570   Prob(JB):                        0.312
Kurtosis:                       3.835   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [196]:
# Get the Model Coefficients:
Paid_Social_Coef = result.params[0]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                       Coef
Paid_Social_Spent  0.001231


In [197]:
df1_Weekly['Paid_Social_Att_OrganicSearch'] = (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)

In [199]:
print(df1_Weekly['Paid_Social_Att_OrganicSearch'].sum())
print(df1_Weekly['TV_Att_OrganicSearch'].sum())
print(df1_Weekly['Organic_Search'].sum())

12555.43483406953
22185.835566469803
35019


In [206]:

df1_Weekly['Paid_Social_Att_OrganicSearch'] = ((df1_Weekly['Organic_Search'] - df1_Weekly['TV_Att_OrganicSearch']))

In [207]:
# Create a New Data Frame to Store Attributed Organic Search Orders:

df_Organic_Search = df1_Weekly[['Date_W', 'TV_Att_OrganicSearch',
                             'Paid_Social_Att_OrganicSearch']].rename(columns= {'TV_Att_OrganicSearch': 'TV_attributed',
                                                                        'Paid_Social_Att_OrganicSearch' :'Paid_Social_attributed'})
df_Organic_Search

,Date_W,TV_attributed,Paid_Social_attributed
0,2019-04-01,232.487103,137.512897
1,2019-04-08,1278.880149,1293.119851
2,2019-04-15,1357.493963,1514.506037
3,2019-04-22,1342.536043,205.463957
4,2019-04-29,1273.142695,215.857305
5,2019-05-06,1399.286909,70.713091
6,2019-05-13,974.850467,585.149533
7,2019-05-20,769.818376,519.181624
8,2019-05-27,765.326257,602.673743
9,2019-06-03,894.009812,352.990188


In [208]:
df_Organic_Search['Channel'] = 'Organic_Search'

df_Organic_Search['Affiliate_attributed'] = 0
df_Organic_Search['All_Other_Paid_attributed'] = 0
df_Organic_Search['Display_attributed'] = 0
df_Organic_Search['Email_attributed'] = 0
df_Organic_Search['SearchBrand_attributed'] = 0
df_Organic_Search['Search_NonBrand_attributed'] = 0
df_Organic_Search['ShoppingAds_attributed'] = 0
#df_Organic_Search['Paid_Social_attributed'] = 0 
#df_Organic_Search['TV_attributed'] = 0 
df_Organic_Search['CTV_attributed'] = 0 

In [209]:
# Paid Search Attributed Orders:

df_Organic_Search = df_Organic_Search[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_Organic_Search.head(3)

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Organic_Search,232.487103,0,137.512897,0,0,0,0,0,0,0
1,2019-04-08,Organic_Search,1278.880149,0,1293.119851,0,0,0,0,0,0,0
2,2019-04-15,Organic_Search,1357.493963,0,1514.506037,0,0,0,0,0,0,0


In [210]:
df_Organic_Search.sum()

Channel                       Organic_SearchOrganic_SearchOrganic_SearchOrga...
TV_attributed                                                           22185.8
CTV_attributed                                                                0
Paid_Social_attributed                                                  12833.2
Affiliate_attributed                                                          0
All_Other_Paid_attributed                                                     0
Display_attributed                                                            0
Email_attributed                                                              0
SearchBrand_attributed                                                        0
Search_NonBrand_attributed                                                    0
ShoppingAds_attributed                                                        0
dtype: object

In [212]:
df_Organic_Search.iloc[:,2:12].sum().sum()

35019.0

In [213]:
# Total TV order Attribution is higher when we combined Paid and Organic Search vs. when we run the regression on each one seperately:
41469.738341

63655.53834099999

# Direct Search Attribution:

In [37]:
y = df1_Weekly[['Direct']].sum(axis=1)
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.994
Model:                            OLS   Adj. R-squared (uncentered):              0.994
Method:                 Least Squares   F-statistic:                              1435.
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    4.28e-28
Time:                        09:56:17   Log-Likelihood:                         -175.21
No. Observations:                  28   AIC:                                      356.4
Df Residuals:                      25   BIC:                                      360.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0015      0.000      6.221      0.000       0.001       0.002
CTV_Spent      -0.0002      0.001     -0.245      0.808      -0.002       0.001
nonTV_Spent     0.0015      0.000      6.934      0.000       0.001       0.002
==============================================================================
Omnibus:                        3.476   Durbin-Watson:                   1.527
Prob(Omnibus):                  0.176   Jarque-Bera (JB):                1.944
Skew:                          -0.527   Prob(JB):                        0.378
Kurtosis:                       3.745   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [38]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                 Coef
TV_Spent     0.001484
CTV_Spent   -0.000161
nonTV_Spent  0.001505


In [39]:
# Predicting the Total Number of Paid_Search Orders using the regression result:

df1_Weekly['Direct_Search_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + 
                                  (df1_Weekly['CTV_Spent']* CTV_Coef)+ 
                                  (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['Direct_Search_Pred'].sum()

43955.06266915853

In [40]:
# Create the two new columns for TV and CTV Attributed Orders based on Paid Search:

df1_Weekly['TV_Att_DirectSearch'] = (df1_Weekly['TV_Spent']* TV_Coef)
df1_Weekly['CTV_Att_DirectSearch'] = (df1_Weekly['CTV_Spent']* CTV_Coef)

In [41]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['TV_Att_DirectSearch'] = ((df1_Weekly['TV_Att_DirectSearch']/df1_Weekly['Direct_Search_Pred'])*y)
df1_Weekly['CTV_Att_DirectSearch'] = ((df1_Weekly['CTV_Att_DirectSearch']/df1_Weekly['Direct_Search_Pred'])*y)


In [42]:
 # Calculate the residuals:
df1_Weekly["Direct_Search_Res"] = y - df1_Weekly['TV_Att_DirectSearch'] - df1_Weekly['CTV_Att_DirectSearch']
df1_Weekly["Direct_Search_Res"].sum()

26883.00778116752

In [43]:
(df1_Weekly[['TV_Att_DirectSearch','CTV_Att_DirectSearch']]).sum().sum()

17435.992218832485

In [44]:
# Sanity Check:
y.sum() - df1_Weekly["Direct_Search_Res"].sum()

17435.99221883248

In [45]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Direct_Search_Res"]
X1= df1_Weekly[['Affiliate_Spent',
                'Email_Spent',
               'Paid_Social_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:      Direct_Search_Res   R-squared (uncentered):                   0.994
Model:                            OLS   Adj. R-squared (uncentered):              0.993
Method:                 Least Squares   F-statistic:                              1368.
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    7.71e-28
Time:                        09:56:22   Log-Likelihood:                         -162.00
No. Observations:                  28   AIC:                                      330.0
Df Residuals:                      25   BIC:                                      334.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Affiliate_Spent       0.0135      0.002      5.853      0.000       0.009       0.018
Email_Spent           0.0109      0.003      4.168      0.000       0.006       0.016
Paid_Social_Spent     0.0012      0.000      4.776      0.000       0.001       0.002
==============================================================================
Omnibus:                        1.109   Durbin-Watson:                   1.653
Prob(Omnibus):                  0.574   Jarque-Bera (JB):                0.702
Skew:                          -0.386   Prob(JB):                        0.704
Kurtosis:                       2.921   Cond. No.                         80.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
# Get the Model Coefficients:
Affiliate_Coef = result.params[0]
Email_Coef = result.params[1]
Paid_Social_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                       Coef
Affiliate_Spent    0.013496
Email_Spent        0.010921
Paid_Social_Spent  0.001204


In [47]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Direct_Res_Pred'] = ((df1_Weekly['Affiliate_Spent']* Affiliate_Coef) + 
                                  (df1_Weekly['Email_Spent']* Email_Coef)+
                                 (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)
                                ) 
df1_Weekly['Direct_Res_Pred'].sum()

26801.658072418857

In [48]:
# Create the two new columns for Attributed Orders based on Direct Search:

df1_Weekly['Affiliate_Att_Direct'] = (df1_Weekly['Affiliate_Spent']* Affiliate_Coef) 
df1_Weekly['Email_Att_Direct'] = (df1_Weekly['Email_Spent']* Email_Coef)
df1_Weekly['Paid_Social_Att_Direct'] = (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)

In [49]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['Affiliate_Att_Direct'] = ((df1_Weekly['Affiliate_Att_Direct']/df1_Weekly['Direct_Res_Pred'])*df1_Weekly['Direct_Search_Res'])

df1_Weekly['Email_Att_Direct'] = ((df1_Weekly['Email_Att_Direct']/df1_Weekly['Direct_Res_Pred'])*df1_Weekly['Direct_Search_Res'])

df1_Weekly['Paid_Social_Att_Direct'] = ((df1_Weekly['Paid_Social_Att_Direct']/df1_Weekly['Direct_Res_Pred'])*df1_Weekly['Direct_Search_Res'])



In [50]:
print(f"Direct_Search_Res_Pred: {df1_Weekly['Direct_Res_Pred'].sum()} \n\
Direct_Search_Res: {df1_Weekly['Direct_Search_Res'].sum()} \n\
{df1_Weekly[['Affiliate_Att_Direct','Email_Att_Direct', 'Paid_Social_Att_Direct']].sum()} \n\
Total_Residuals: {df1_Weekly[['Affiliate_Att_Direct','Email_Att_Direct', 'Paid_Social_Att_Direct']].sum().sum()}" )

Direct_Search_Res_Pred: 26801.658072418857 
Direct_Search_Res: 26883.00778116752 
Affiliate_Att_Direct       9108.235849
Email_Att_Direct           5458.687823
Paid_Social_Att_Direct    12316.084110
dtype: float64 
Total_Residuals: 26883.007781167507


In [51]:
# Create a New Data Frame to Store Attributed Direct Orders:

df_Direct_Search = df1_Weekly[['Date_W', 'TV_Att_DirectSearch','CTV_Att_DirectSearch',
                             'Affiliate_Att_Direct', 'Email_Att_Direct',
                             'Paid_Social_Att_Direct']].rename(columns= {'TV_Att_DirectSearch': 'TV_attributed',
                                'CTV_Att_DirectSearch':'CTV_attributed',
                                'Affiliate_Att_Direct': 'Affiliate_attributed',
                                'Email_Att_Direct':'Email_attributed' ,                                      
                                'Paid_Social_Att_Direct' :'Paid_Social_attributed'})
df_Direct_Search.head(2)

,Date_W,TV_attributed,CTV_attributed,Affiliate_attributed,Email_attributed,Paid_Social_attributed
0,2019-04-01,180.419429,-4.422089,40.848681,90.634012,103.519966
1,2019-04-08,1138.126730,-16.856861,363.849537,459.779878,743.100716


In [52]:
df_Direct_Search['Channel'] = 'Direct'

#df_Direct_Search['Affiliate_attributed'] = 0
df_Direct_Search['All_Other_Paid_attributed'] = 0
df_Direct_Search['Display_attributed'] = 0
#df_Direct_Search['Email_attributed'] = 0
df_Direct_Search['SearchBrand_attributed'] = 0
df_Direct_Search['Search_NonBrand_attributed'] = 0
df_Direct_Search['ShoppingAds_attributed'] = 0
#df_Direct_Search['Paid_Social_attributed'] = 0 
#df_Direct_Search['TV_attributed'] = 0 
#df_Direct_Search['CTV_attributed'] = 0 

In [53]:
# Direct Attributed Orders:

df_Direct_Search = df_Direct_Search[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_Direct_Search.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Direct,180.419429,-4.422089,103.519966,40.848681,0,0,90.634012,0,0,0
1,2019-04-08,Direct,1138.126730,-16.856861,743.100716,363.849537,0,0,459.779878,0,0,0
2,2019-04-15,Direct,1105.082898,-33.089285,808.853038,309.321312,0,0,474.832037,0,0,0
3,2019-04-22,Direct,936.648490,-27.353933,541.282764,243.419240,0,0,370.003438,0,0,0
4,2019-04-29,Direct,1016.820337,-37.380049,461.853106,256.591697,0,0,415.114910,0,0,0


In [54]:
df_Direct_Search.iloc[:,2:12].sum()

TV_attributed                 18135.393423
CTV_attributed                 -699.401204
Paid_Social_attributed        12316.084110
Affiliate_attributed           9108.235849
All_Other_Paid_attributed         0.000000
Display_attributed                0.000000
Email_attributed               5458.687823
SearchBrand_attributed            0.000000
Search_NonBrand_attributed        0.000000
ShoppingAds_attributed            0.000000
dtype: float64

In [55]:
df_Direct_Search.iloc[:,2:12].sum().sum()

44318.99999999999

# Paid and Organic Social Attribution

In [1]:
21432 + 2982

24414

In [56]:
y = df1_Weekly[['Organic_Social', 'Paid_Social']].sum(axis=1)
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.979
Model:                            OLS   Adj. R-squared (uncentered):              0.976
Method:                 Least Squares   F-statistic:                              387.8
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    4.43e-21
Time:                        09:56:39   Log-Likelihood:                         -177.88
No. Observations:                  28   AIC:                                      361.8
Df Residuals:                      25   BIC:                                      365.8
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0008      0.000      3.119      0.005       0.000       0.001
CTV_Spent      -0.0020      0.001     -2.759      0.011      -0.003      -0.001
nonTV_Spent     0.0013      0.000      5.528      0.000       0.001       0.002
==============================================================================
Omnibus:                        2.200   Durbin-Watson:                   1.035
Prob(Omnibus):                  0.333   Jarque-Bera (JB):                0.961
Skew:                           0.155   Prob(JB):                        0.618
Kurtosis:                       3.853   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                 Coef
TV_Spent     0.000818
CTV_Spent   -0.001997
nonTV_Spent  0.001319


In [58]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Paid_Organic_Social_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + 
                                  (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['Paid_Organic_Social_Pred'].sum()

33302.18453080027

In [59]:
# Create a new columns for TV Attributed Orders based on Paid & Organic Social:

df1_Weekly['TV_Att_PaidOrganicSocial'] = (df1_Weekly['TV_Spent']* TV_Coef)

In [60]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['TV_Att_PaidOrganicSocial'] = ((df1_Weekly['TV_Att_PaidOrganicSocial']/df1_Weekly['Paid_Organic_Social_Pred'])*y)


In [61]:
# Calculate the residuals:
df1_Weekly["PaidOrganic_Social_Res"] = y - df1_Weekly['TV_Att_PaidOrganicSocial']
df1_Weekly["PaidOrganic_Social_Res"].sum()

17136.433263829887

In [62]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["PaidOrganic_Social_Res"]
X1= df1_Weekly[['Display_Spent',
               'Paid_Social_Spent',
                'Search_Brand_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                   OLS Regression Results                                  
===========================================================================================
Dep. Variable:     PaidOrganic_Social_Res   R-squared (uncentered):                   0.982
Model:                                OLS   Adj. R-squared (uncentered):              0.980
Method:                     Least Squares   F-statistic:                              461.2
Date:                    Thu, 23 Jul 2020   Prob (F-statistic):                    5.30e-22
Time:                            09:56:44   Log-Likelihood:                         -165.65
No. Observations:                      28   AIC:                                      337.3
Df Residuals:                          25   BIC:                                      341.3
Df Model:                               3                                                  
Covariance Type:                nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Display_Spent          0.0104      0.003      3.210      0.004       0.004       0.017
Paid_Social_Spent      0.0009      0.000      4.212      0.000       0.000       0.001
Search_Brand_Spent     0.0041      0.002      2.466      0.021       0.001       0.007
==============================================================================
Omnibus:                        0.727   Durbin-Watson:                   1.092
Prob(Omnibus):                  0.695   Jarque-Bera (JB):                0.083
Skew:                           0.022   Prob(JB):                        0.960
Kurtosis:                       3.262   Cond. No.                         71.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
# Get the Model Coefficients:
Display_Coef = result.params[0]
Paid_Social_Coef = result.params[1]
Search_Brand_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                        Coef
Display_Spent       0.010399
Paid_Social_Spent   0.000945
Search_Brand_Spent  0.004086


In [64]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['PaidOrganic_Social_Res_Pred'] = ((df1_Weekly['Display_Spent']* Display_Coef)  + 
                                  (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef) + 
                                     (df1_Weekly['Search_Brand_Spent']* Search_Brand_Coef)) 
df1_Weekly['PaidOrganic_Social_Res_Pred'].sum()

17130.221793625675

In [65]:
# Create the two new columns for Attributed Orders based on Paid & Organic Social:

df1_Weekly['Display_Att_PaidOrganic_Socail'] = (df1_Weekly['Display_Spent']* Display_Coef) 

df1_Weekly['Paid_Social_Att_PaidOrganic_Social'] = (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)

df1_Weekly['Search_Brand_Att_PaidOrganic_Social'] = (df1_Weekly['Search_Brand_Spent']* Search_Brand_Coef)


In [66]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['Display_Att_PaidOrganic_Socail'] = (df1_Weekly['Display_Att_PaidOrganic_Socail']/df1_Weekly['PaidOrganic_Social_Res_Pred'])* y

df1_Weekly['Paid_Social_Att_PaidOrganic_Social'] = ((df1_Weekly['Paid_Social_Att_PaidOrganic_Social']/df1_Weekly['PaidOrganic_Social_Res_Pred'])*y)

df1_Weekly['Search_Brand_Att_PaidOrganic_Social'] =(df1_Weekly['Search_Brand_Att_PaidOrganic_Social']/df1_Weekly['PaidOrganic_Social_Res_Pred'])* y

In [67]:
print(f"PaidOrganic_Socail_Res_Pred: {df1_Weekly['PaidOrganic_Social_Res_Pred'].sum()} \n\
PaidOrganic_Social_Res: {df1_Weekly['PaidOrganic_Social_Res'].sum()} \n\
{df1_Weekly[['Display_Att_PaidOrganic_Socail','Paid_Social_Att_PaidOrganic_Social', 'Search_Brand_Att_PaidOrganic_Social']].sum()} \n\
Total_Residuals: {df1_Weekly[['Display_Att_PaidOrganic_Socail','Paid_Social_Att_PaidOrganic_Social', 'Search_Brand_Att_PaidOrganic_Social']].sum().sum()}" )

PaidOrganic_Socail_Res_Pred: 17130.221793625675 
PaidOrganic_Social_Res: 17136.433263829887 
Display_Att_PaidOrganic_Socail         2791.281000
Paid_Social_Att_PaidOrganic_Social     9648.961826
Search_Brand_Att_PaidOrganic_Social    4696.190438
dtype: float64 
Total_Residuals: 17136.433263829887


In [68]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_PaidOrganic_Social = df1_Weekly[['Date_W', 
                                'TV_Att_PaidOrganicSocial',
                                 'Display_Att_PaidOrganic_Socail', 
                                 'Paid_Social_Att_PaidOrganic_Social', 
                        'Search_Brand_Att_PaidOrganic_Social']].rename(columns= {'TV_Att_PaidOrganicSocial': 'TV_attributed',
                            'Display_Att_PaidOrganic_Socail': 'Display_attributed', 
                            'Paid_Social_Att_PaidOrganic_Social' :'Paid_Social_attributed',
                            'Search_Brand_Att_PaidOrganic_Social': 'SearchBrand_attributed' })
df_PaidOrganic_Social.head(2)

,Date_W,TV_attributed,Display_attributed,Paid_Social_attributed,SearchBrand_attributed
0,2019-04-01,84.658503,46.040824,89.973270,39.327403
1,2019-04-08,546.581471,278.630525,638.340226,279.447778


In [69]:
df_PaidOrganic_Social['Channel'] = 'Paid_Organic_Search'

df_PaidOrganic_Social['Affiliate_attributed'] = 0
df_PaidOrganic_Social['All_Other_Paid_attributed'] = 0
#df_PaidOrganic_Social['Display_attributed'] = 0
df_PaidOrganic_Social['Email_attributed'] = 0
#df_PaidOrganic_Social['SearchBrand_attributed'] = 0
df_PaidOrganic_Social['Search_NonBrand_attributed'] = 0
df_PaidOrganic_Social['ShoppingAds_attributed'] = 0
#df_PaidOrganic_Social['Paid_Social_attributed'] = 0 
#df_PaidOrganic_Social['TV_attributed'] = 0 
df_PaidOrganic_Social['CTV_attributed'] = 0 

In [70]:
# Paid Search Attributed Orders:

df_PaidOrganic_Social = df_PaidOrganic_Social[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_PaidOrganic_Social.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Paid_Organic_Search,84.658503,0,89.973270,0,0,46.040824,0,39.327403,0,0
1,2019-04-08,Paid_Organic_Search,546.581471,0,638.340226,0,0,278.630525,0,279.447778,0,0
2,2019-04-15,Paid_Organic_Search,588.972770,0,728.719984,0,0,335.166882,0,286.140364,0,0
3,2019-04-22,Paid_Organic_Search,460.428633,0,453.735163,0,0,194.695683,0,254.140522,0,0
4,2019-04-29,Paid_Organic_Search,413.321877,0,314.330848,0,0,181.770050,0,236.577226,0,0


In [71]:
df_PaidOrganic_Social.iloc[:,2:12].sum()

TV_attributed                 7277.566736
CTV_attributed                   0.000000
Paid_Social_attributed        9648.961826
Affiliate_attributed             0.000000
All_Other_Paid_attributed        0.000000
Display_attributed            2791.281000
Email_attributed                 0.000000
SearchBrand_attributed        4696.190438
Search_NonBrand_attributed       0.000000
ShoppingAds_attributed           0.000000
dtype: float64

In [72]:
df_PaidOrganic_Social.iloc[:,2:12].sum().sum()

24414.0

# Shopping Ads Attributed Orders:

In [73]:
y = df1_Weekly['Shopping_Ads']
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           Shopping_Ads   R-squared (uncentered):                   0.982
Model:                            OLS   Adj. R-squared (uncentered):              0.980
Method:                 Least Squares   F-statistic:                              465.4
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    4.74e-22
Time:                        09:57:10   Log-Likelihood:                         -169.69
No. Observations:                  28   AIC:                                      345.4
Df Residuals:                      25   BIC:                                      349.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0008      0.000      4.235      0.000       0.000       0.001
CTV_Spent       0.0001      0.001      0.210      0.836      -0.001       0.001
nonTV_Spent     0.0006      0.000      3.182      0.004       0.000       0.001
==============================================================================
Omnibus:                        0.306   Durbin-Watson:                   0.856
Prob(Omnibus):                  0.858   Jarque-Bera (JB):                0.479
Skew:                          -0.161   Prob(JB):                        0.787
Kurtosis:                       2.445   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [74]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                 Coef
TV_Spent     0.000829
CTV_Spent    0.000113
nonTV_Spent  0.000567


In [75]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['ShoppingAds_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + (df1_Weekly['CTV_Spent']* CTV_Coef)+
                                  (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['ShoppingAds_Pred'].sum()

20586.459240544384

In [76]:
df1_Weekly['Shopping_Ads'].sum()

21001

In [77]:
df1_Weekly['TV_Att_ShoppingAds'] = (df1_Weekly['TV_Spent']* TV_Coef)
df1_Weekly['TV_Att_ShoppingAds'].sum()


10048.090608307119

In [78]:
# Rescale the predicted Orders based on total Paid Shopping Ads Orders:

df1_Weekly['TV_Att_ShoppingAds'] = ((df1_Weekly['TV_Att_ShoppingAds']/df1_Weekly['ShoppingAds_Pred'])*df1_Weekly['Shopping_Ads'])
df1_Weekly['TV_Att_ShoppingAds'].sum()


10268.214707371257

In [79]:
df1_Weekly['Residuals_ShoppingAds'] = y - df1_Weekly['TV_Att_ShoppingAds']

In [80]:
df1_Weekly['Residuals_ShoppingAds'].sum()

10732.785292628743

In [81]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Residuals_ShoppingAds"]
X1= df1_Weekly[['Affiliate_Spent', 
                'All_Other_Paid_Spent', 
                'Display_Spent',
               'Email_Spent',
               'Paid_Social_Spent',
                'Search_Brand_Spent',
               'Search_NonBrand_Spent',
               'Shopping_Ads_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     Residuals_ShoppingAds   R-squared (uncentered):                   0.992
Model:                               OLS   Adj. R-squared (uncentered):              0.989
Method:                    Least Squares   F-statistic:                              316.1
Date:                   Thu, 23 Jul 2020   Prob (F-statistic):                    2.48e-19
Time:                           09:57:16   Log-Likelihood:                         -139.70
No. Observations:                     28   AIC:                                      295.4
Df Residuals:                         20   BIC:                                      306.1
Df Model:                              8                                                  
Covariance Type:               nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent           0.0065      0.002      2.709      0.014       0.001       0.011
All_Other_Paid_Spent      0.0013      0.001      1.449      0.163      -0.001       0.003
Display_Spent            -0.0005      0.004     -0.141      0.890      -0.008       0.007
Email_Spent               0.0050      0.003      1.662      0.112      -0.001       0.011
Paid_Social_Spent         0.0007      0.000      3.370      0.003       0.000       0.001
Search_Brand_Spent        0.0006      0.001      0.423      0.677      -0.002       0.003
Search_NonBrand_Spent    -0.0004      0.003     -0.139      0.891      -0.006       0.005
Shopping_Ads_Spent       -0.0042      0.002     -2.030      0.056      -0.009       0.000
==============================================================================
Omnibus:                        1.192   Durbin-Watson:                   1.862
Prob(Omnibus):                  0.551   Jarque-Bera (JB):                0.740
Skew:                          -0.397   Prob(JB):                        0.691
Kurtosis:                       2.945   Cond. No.                         202.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [82]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Residuals_ShoppingAds"]
X1= df1_Weekly[['Affiliate_Spent', 
               'Email_Spent',
               'Paid_Social_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                  
==========================================================================================
Dep. Variable:     Residuals_ShoppingAds   R-squared (uncentered):                   0.989
Model:                               OLS   Adj. R-squared (uncentered):              0.988
Method:                    Least Squares   F-statistic:                              751.2
Date:                   Thu, 23 Jul 2020   Prob (F-statistic):                    1.30e-24
Time:                           09:57:17   Log-Likelihood:                         -144.39
No. Observations:                     28   AIC:                                      294.8
Df Residuals:                         25   BIC:                                      298.8
Df Model:                              3                                                  
Covariance Type:               nonrobust                                                  
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Affiliate_Spent       0.0064      0.001      5.177      0.000       0.004       0.009
Email_Spent           0.0030      0.001      2.175      0.039       0.000       0.006
Paid_Social_Spent     0.0005      0.000      3.572      0.001       0.000       0.001
==============================================================================
Omnibus:                        5.879   Durbin-Watson:                   1.326
Prob(Omnibus):                  0.053   Jarque-Bera (JB):                4.089
Skew:                          -0.713   Prob(JB):                        0.129
Kurtosis:                       4.213   Cond. No.                         80.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [83]:
# Get the Model Coefficients:
Affiliate_Coef = result.params[0]
Email_Coef = result.params[1]
Paid_Social_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                       Coef
Affiliate_Spent    0.006366
Email_Spent        0.003039
Paid_Social_Spent  0.000480


In [84]:
# Predicting the Total Number of shopping Ads Orders using the regression result:

df1_Weekly['ShoppingAds_Res_Pred'] = ((df1_Weekly['Affiliate_Spent']* Affiliate_Coef) + 
                                      (df1_Weekly['Email_Spent']* Email_Coef) +
                                      (df1_Weekly['Paid_Social_Spent']* Paid_Social_Coef)) 
df1_Weekly['ShoppingAds_Res_Pred'].sum()

10694.568574959867

In [ ]:
10759.321528121078

In [85]:
# Create the two new columns for Attributed Orders based on Paid Search:

df1_Weekly['Affiliate_Att_ShoppingAds'] = (df1_Weekly['Affiliate_Spent'] * Affiliate_Coef) 
df1_Weekly['Email_Att_ShoppingAds'] = (df1_Weekly['Email_Spent'] * Email_Coef)
df1_Weekly['Paid_Social_Att_ShoppingAds'] = (df1_Weekly['Paid_Social_Spent'] * Paid_Social_Coef)


In [86]:
# Rescale the Columns based on Actual Orders:

df1_Weekly['Affiliate_Att_ShoppingAds'] = ((df1_Weekly['Affiliate_Att_ShoppingAds']/df1_Weekly['ShoppingAds_Res_Pred'])*y)
df1_Weekly['Email_Att_ShoppingAds'] = ((df1_Weekly['Email_Att_ShoppingAds']/df1_Weekly['ShoppingAds_Res_Pred'])*y)
df1_Weekly['Paid_Social_Att_ShoppingAds'] = ((df1_Weekly['Paid_Social_Att_ShoppingAds']/df1_Weekly['ShoppingAds_Res_Pred'])*y)



In [87]:
print(f"ShoppingAds_Res_Pred: {df1_Weekly['ShoppingAds_Res_Pred'].sum()} \n\
ShoppingAds_Search_Res: {df1_Weekly['Residuals_ShoppingAds'].sum()} \n\
{df1_Weekly[['Affiliate_Att_ShoppingAds','Email_Att_ShoppingAds', 'Paid_Social_Att_ShoppingAds']].sum()} \n\
Total_Residuals: {df1_Weekly[['Affiliate_Att_ShoppingAds','Email_Att_ShoppingAds', 'Paid_Social_Att_ShoppingAds']].sum().sum()}" )

ShoppingAds_Res_Pred: 10694.568574959867 
ShoppingAds_Search_Res: 10732.785292628743 
Affiliate_Att_ShoppingAds      4305.236930
Email_Att_ShoppingAds          1516.270976
Paid_Social_Att_ShoppingAds    4911.277387
dtype: float64 
Total_Residuals: 10732.785292628745


In [88]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_ShoppingAds = df1_Weekly[['Date_W', 'TV_Att_ShoppingAds',
                             'Affiliate_Att_ShoppingAds', 'Email_Att_ShoppingAds',
                             'Paid_Social_Att_ShoppingAds']].rename(columns= {'TV_Att_ShoppingAds': 'TV_attributed',
                                'Affiliate_Att_ShoppingAds': 'Affiliate_attributed', 
                                  'Email_Att_ShoppingAds': 'Email_attributed',                                          
                                'Paid_Social_Att_ShoppingAds' :'Paid_Social_attributed'})
df_ShoppingAds.head(2)

,Date_W,TV_attributed,Affiliate_attributed,Email_attributed,Paid_Social_attributed
0,2019-04-01,103.727945,21.175574,27.721518,45.374963
1,2019-04-08,667.841038,182.049357,135.733152,314.376453


In [89]:
df_ShoppingAds['Channel'] = 'Shopping_Ads'

#df_ShoppingAds['Affiliate_attributed'] = 0
df_ShoppingAds['All_Other_Paid_attributed'] = 0
df_ShoppingAds['Display_attributed'] = 0
#df_ShoppingAds['Email_attributed'] = 0
df_ShoppingAds['SearchBrand_attributed'] = 0
df_ShoppingAds['Search_NonBrand_attributed'] = 0
df_ShoppingAds['ShoppingAds_attributed'] = 0
#df_ShoppingAds['Paid_Social_attributed'] = 0 
#df_ShoppingAds['TV_attributed'] = 0 
df_ShoppingAds['CTV_attributed'] = 0 

In [90]:
# Paid Search Attributed Orders:

df_ShoppingAds = df_ShoppingAds[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_ShoppingAds.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Shopping_Ads,103.727945,0,45.374963,21.175574,0,0,27.721518,0,0,0
1,2019-04-08,Shopping_Ads,667.841038,0,314.376453,182.049357,0,0,135.733152,0,0,0
2,2019-04-15,Shopping_Ads,648.465099,0,351.537734,158.992693,0,0,144.004475,0,0,0
3,2019-04-22,Shopping_Ads,518.275481,0,223.871124,119.067649,0,0,106.785746,0,0,0
4,2019-04-29,Shopping_Ads,496.749568,0,173.470955,113.980483,0,0,108.798994,0,0,0


In [91]:
df_ShoppingAds.iloc[:,2:12].sum()

TV_attributed                 10268.214707
CTV_attributed                    0.000000
Paid_Social_attributed         4911.277387
Affiliate_attributed           4305.236930
All_Other_Paid_attributed         0.000000
Display_attributed                0.000000
Email_attributed               1516.270976
SearchBrand_attributed            0.000000
Search_NonBrand_attributed        0.000000
ShoppingAds_attributed            0.000000
dtype: float64

In [92]:
df_ShoppingAds.iloc[:,2:12].sum().sum()

21001.000000000004

# Email Attribution Orders

In [93]:
# Direct as the Dependent Variable on Total Orders:

y = df1_Weekly['Email'] 
X= df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of Display on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

# Only Non_TV Spent is significat

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Email   R-squared (uncentered):                   0.964
Model:                            OLS   Adj. R-squared (uncentered):              0.960
Method:                 Least Squares   F-statistic:                              223.3
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    3.61e-18
Time:                        09:57:38   Log-Likelihood:                         -171.15
No. Observations:                  28   AIC:                                      348.3
Df Residuals:                      25   BIC:                                      352.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent       -0.0004      0.000     -2.111      0.045      -0.001   -1.06e-05
CTV_Spent      -0.0008      0.001     -1.403      0.173      -0.002       0.000
nonTV_Spent     0.0013      0.000      7.003      0.000       0.001       0.002
==============================================================================
Omnibus:                        5.276   Durbin-Watson:                   0.942
Prob(Omnibus):                  0.072   Jarque-Bera (JB):                4.117
Skew:                           0.936   Prob(JB):                        0.128
Kurtosis:                       3.161   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [94]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Email"]
X1= df1_Weekly[['Email',
               'Shopping_Ads_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  Email   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          7.218e+32
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                        0.00
Time:                        09:57:39   Log-Likelihood:                          805.63
No. Observations:                  28   AIC:                                     -1607.
Df Residuals:                      26   BIC:                                     -1605.
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Email                  1.0000   1.34e-16   7.45e+15      0.000       1.000       1.000
Shopping_Ads_Spent  1.735e-18    2.3e-18      0.754      0.458   -2.99e-18    6.46e-18
==============================================================================
Omnibus:                        4.549   Durbin-Watson:                   0.357
Prob(Omnibus):                  0.103   Jarque-Bera (JB):                1.694
Skew:                           0.126   Prob(JB):                        0.429
Kurtosis:                       1.822   Cond. No.                         298.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [95]:
# All Orders go to Email

In [96]:
# Create a New Data Frame to Store Attributed Direct Orders:

df_Email = df1_Weekly[['Date_W', 'Email']].rename(columns= {'Email':'Email_attributed' })
df_Email.head(2)

,Date_W,Email_attributed
0,2019-04-01,161
1,2019-04-08,706


In [97]:
df_Email['Channel'] = 'Email'

df_Email['Affiliate_attributed'] = 0
df_Email['All_Other_Paid_attributed'] = 0
df_Email['Display_attributed'] = 0
#df_Email['Email_attributed'] = 0
df_Email['SearchBrand_attributed'] = 0
df_Email['Search_NonBrand_attributed'] = 0
df_Email['ShoppingAds_attributed'] = 0
df_Email['Paid_Social_attributed'] = 0 
df_Email['TV_attributed'] = 0 
df_Email['CTV_attributed'] = 0 

In [98]:
# Direct Attributed Orders:

df_Email = df_Email[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_Email.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Email,0,0,0,0,0,0,161,0,0,0
1,2019-04-08,Email,0,0,0,0,0,0,706,0,0,0
2,2019-04-15,Email,0,0,0,0,0,0,731,0,0,0
3,2019-04-22,Email,0,0,0,0,0,0,656,0,0,0
4,2019-04-29,Email,0,0,0,0,0,0,759,0,0,0


In [99]:
df_Email.iloc[:,2:12].sum()

TV_attributed                     0
CTV_attributed                    0
Paid_Social_attributed            0
Affiliate_attributed              0
All_Other_Paid_attributed         0
Display_attributed                0
Email_attributed              14387
SearchBrand_attributed            0
Search_NonBrand_attributed        0
ShoppingAds_attributed            0
dtype: int64

In [100]:
df_Email.iloc[:,2:12].sum().sum()

14387

# Affiliate Attributed Orders

In [101]:
df1_Weekly['Affiliate'].sum()

14027

In [102]:
y = df1_Weekly['Affiliate']
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              Affiliate   R-squared (uncentered):                   0.875
Model:                            OLS   Adj. R-squared (uncentered):              0.860
Method:                 Least Squares   F-statistic:                              58.22
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    2.03e-11
Time:                        09:57:48   Log-Likelihood:                         -185.34
No. Observations:                  28   AIC:                                      376.7
Df Residuals:                      25   BIC:                                      380.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0004      0.000      1.275      0.214      -0.000       0.001
CTV_Spent       0.0021      0.001      2.269      0.032       0.000       0.004
nonTV_Spent -8.993e-05      0.000     -0.289      0.775      -0.001       0.001
==============================================================================
Omnibus:                        3.465   Durbin-Watson:                   0.558
Prob(Omnibus):                  0.177   Jarque-Bera (JB):                2.659
Skew:                          -0.754   Prob(JB):                        0.265
Kurtosis:                       2.946   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [103]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                 Coef
TV_Spent     0.000436
CTV_Spent    0.002143
nonTV_Spent -0.000090


In [104]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['CTV_Att_Affiliate'] = (df1_Weekly['CTV_Spent']* CTV_Coef) 
df1_Weekly['CTV_Att_Affiliate'].sum()

9213.051701643626

In [105]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Affiliate_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + (df1_Weekly['CTV_Spent']* CTV_Coef)+
                                      (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['Affiliate_Pred'].sum()

12906.023568034425

In [106]:
df1_Weekly['CTV_Att_Affiliate']  = ((df1_Weekly['CTV_Spent']* CTV_Coef) /df1_Weekly['Affiliate_Pred']) * y
df1_Weekly['CTV_Att_Affiliate'].sum()

9988.257369402165

In [107]:
df1_Weekly['Affiliate_Res'] = y - df1_Weekly['CTV_Att_Affiliate']
df1_Weekly['Affiliate_Res'].sum()

4038.7426305978347

In [108]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Affiliate_Res"]
X1= df1_Weekly[['Affiliate_Spent', 
                'Search_Brand_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Affiliate_Res   R-squared (uncentered):                   0.911
Model:                            OLS   Adj. R-squared (uncentered):              0.905
Method:                 Least Squares   F-statistic:                              133.8
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    2.06e-14
Time:                        09:57:51   Log-Likelihood:                         -146.77
No. Observations:                  28   AIC:                                      297.5
Df Residuals:                      26   BIC:                                      300.2
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Affiliate_Spent        0.0037      0.001      3.466      0.002       0.002       0.006
Search_Brand_Spent     0.0013      0.001      2.123      0.043    4.18e-05       0.003
==============================================================================
Omnibus:                        1.939   Durbin-Watson:                   0.958
Prob(Omnibus):                  0.379   Jarque-Bera (JB):                0.785
Skew:                           0.287   Prob(JB):                        0.675
Kurtosis:                       3.586   Cond. No.                         6.72
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [109]:
# Get the Model Coefficients
Affiliate_Coef = result.params[0]
Search_Brand_Coef = result.params[1]

print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                        Coef
Affiliate_Spent     0.003748
Search_Brand_Spent  0.001319


In [110]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Affiliate_Res_Pred'] = ((df1_Weekly['Affiliate']* Affiliate_Coef)  + 
                                     (df1_Weekly['Search_Brand_Spent']* Search_Brand_Coef)) 
df1_Weekly['Affiliate_Res_Pred'].sum()

1566.0282253538637

In [111]:
# Create the two new columns for Attributed Orders based on Paid & Organic Social:

df1_Weekly['Affiliate_Att_Affiliate'] = (df1_Weekly['Affiliate']* Affiliate_Coef) 

df1_Weekly['Search_Brand_Att_Affiliate'] = (df1_Weekly['Search_Brand_Spent']* Search_Brand_Coef)

In [112]:
df1_Weekly['Affiliate_Att_Affiliate'] = (df1_Weekly['Affiliate_Att_Affiliate']/df1_Weekly['Affiliate_Res_Pred'] ) *y

df1_Weekly['Search_Brand_Att_Affiliate'] = (df1_Weekly['Search_Brand_Att_Affiliate']/df1_Weekly['Affiliate_Res_Pred'] ) *y

In [113]:
print(f"Affiliate_Res_Pred: {df1_Weekly['Affiliate_Res_Pred'].sum()} \n\
Affiliate_Res: {y.sum()} \n\
{df1_Weekly[['Affiliate_Att_Affiliate','Search_Brand_Att_Affiliate']].sum()} \n\
Total_Residuals: {df1_Weekly[['Affiliate_Att_Affiliate','Search_Brand_Att_Affiliate']].sum().sum()}" )

Affiliate_Res_Pred: 1566.0282253538637 
Affiliate_Res: 4038.7426305978347 
Affiliate_Att_Affiliate        147.619611
Search_Brand_Att_Affiliate    3891.123019
dtype: float64 
Total_Residuals: 4038.7426305978333


In [114]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_Affiliate = df1_Weekly[['Date_W', 
                                'CTV_Att_Affiliate',
                                 'Affiliate_Att_Affiliate', 
                                 'Search_Brand_Att_Affiliate']].rename(columns= {'CTV_Att_Affiliate': 'CTV_attributed',
                            'Affiliate_Att_Affiliate': 'Affiliate_attributed', 
                            'Search_Brand_Att_Affiliate': 'SearchBrand_attributed' })
df_Affiliate.head(2)

,Date_W,CTV_attributed,Affiliate_attributed,SearchBrand_attributed
0,2019-04-01,96.181442,2.795304,61.023254
1,2019-04-08,297.750685,8.452330,311.796986


In [115]:
df_Affiliate['Channel'] = 'Affiliate'

#df_Affiliate['Affiliate_attributed'] = 0
df_Affiliate['All_Other_Paid_attributed'] = 0
df_Affiliate['Display_attributed'] = 0
df_Affiliate['Email_attributed'] = 0
#df_Affiliate['SearchBrand_attributed'] = 0
df_Affiliate['Search_NonBrand_attributed'] = 0
df_Affiliate['ShoppingAds_attributed'] = 0
df_Affiliate['Paid_Social_attributed'] = 0 
df_Affiliate['TV_attributed'] = 0 
#df_Affiliate['CTV_attributed'] = 0 

In [116]:
# Paid Search Attributed Orders:

df_Affiliate = df_Affiliate[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_Affiliate.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Affiliate,0,96.181442,0,2.795304,0,0,0,61.023254,0,0
1,2019-04-08,Affiliate,0,297.750685,0,8.452330,0,0,0,311.796986,0,0
2,2019-04-15,Affiliate,0,359.330480,0,4.758648,0,0,0,182.910872,0,0
3,2019-04-22,Affiliate,0,320.872062,0,3.859922,0,0,0,178.268015,0,0
4,2019-04-29,Affiliate,0,219.062397,0,1.436990,0,0,0,100.500612,0,0


In [117]:
df_Affiliate.iloc[:,2:12].sum()

TV_attributed                    0.000000
CTV_attributed                9988.257369
Paid_Social_attributed           0.000000
Affiliate_attributed           147.619611
All_Other_Paid_attributed        0.000000
Display_attributed               0.000000
Email_attributed                 0.000000
SearchBrand_attributed        3891.123019
Search_NonBrand_attributed       0.000000
ShoppingAds_attributed           0.000000
dtype: float64

In [118]:
df_Affiliate.iloc[:,2:12].sum().sum()

14026.999999999996

# All Other Non Paid Attributions

In [119]:
y = df1_Weekly['All_Other_Non-Paid']
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     All_Other_Non-Paid   R-squared (uncentered):                   0.845
Model:                            OLS   Adj. R-squared (uncentered):              0.826
Method:                 Least Squares   F-statistic:                              45.44
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    2.87e-10
Time:                        09:58:05   Log-Likelihood:                         -161.32
No. Observations:                  28   AIC:                                      328.6
Df Residuals:                      25   BIC:                                      332.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent    -2.832e-05      0.000     -0.195      0.847      -0.000       0.000
CTV_Spent       0.0007      0.000      1.774      0.088      -0.000       0.002
nonTV_Spent     0.0001      0.000      0.896      0.379      -0.000       0.000
==============================================================================
Omnibus:                        2.201   Durbin-Watson:                   0.805
Prob(Omnibus):                  0.333   Jarque-Bera (JB):                0.980
Skew:                           0.042   Prob(JB):                        0.613
Kurtosis:                       3.913   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [120]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))


                 Coef
TV_Spent    -0.000028
CTV_Spent    0.000711
nonTV_Spent  0.000118


In [121]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['All_Other_Non-Paid_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + (df1_Weekly['CTV_Spent']* CTV_Coef)+
                                  (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['All_Other_Non-Paid_Pred'].sum()

4809.467617511396

In [122]:
y.sum()

5221

In [123]:
df1_Weekly['CTV_Att_AllOtherNonPaid'] = (df1_Weekly['CTV_Spent']* CTV_Coef)
df1_Weekly['CTV_Att_AllOtherNonPaid'].sum()

3054.687056124778

In [124]:
df1_Weekly['CTV_Att_AllOtherNonPaid'] = (df1_Weekly['CTV_Att_AllOtherNonPaid']/df1_Weekly['All_Other_Non-Paid_Pred']) * y
df1_Weekly['CTV_Att_AllOtherNonPaid'].sum()

3327.5621244614304

In [125]:
df1_Weekly['AllOtherNonPaid_Res'] = y - df1_Weekly['CTV_Att_AllOtherNonPaid']
df1_Weekly['AllOtherNonPaid_Res'].sum()

1893.4378755385696

In [126]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["AllOtherNonPaid_Res"]
X1= df1_Weekly[['Affiliate_Spent', 
               'Search_NonBrand_Spent',
               'Shopping_Ads_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     AllOtherNonPaid_Res   R-squared (uncentered):                   0.954
Model:                             OLS   Adj. R-squared (uncentered):              0.948
Method:                  Least Squares   F-statistic:                              172.5
Date:                 Thu, 23 Jul 2020   Prob (F-statistic):                    7.94e-17
Time:                         09:58:10   Log-Likelihood:                         -116.46
No. Observations:                   28   AIC:                                      238.9
Df Residuals:                       25   BIC:                                      242.9
Df Model:                            3                                                  
Covariance Type:             nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent           0.0025      0.001      4.624      0.000       0.001       0.004
Search_NonBrand_Spent     0.0002      0.001      0.363      0.720      -0.001       0.001
Shopping_Ads_Spent        0.0002      0.001      0.346      0.732      -0.001       0.002
==============================================================================
Omnibus:                        2.492   Durbin-Watson:                   1.216
Prob(Omnibus):                  0.288   Jarque-Bera (JB):                2.036
Skew:                           0.647   Prob(JB):                        0.361
Kurtosis:                       2.735   Cond. No.                         14.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [127]:
# The rest of the orders go to Affiliate Spent

In [128]:
df1_Weekly['Affiliate_Att_AllOtherNonPaid'] = df1_Weekly["AllOtherNonPaid_Res"]

In [129]:
df1_Weekly[['CTV_Att_AllOtherNonPaid','Affiliate_Att_AllOtherNonPaid']].sum().sum()

5221.0

In [130]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_All_Other_Non_Paid = df1_Weekly[['Date_W','CTV_Att_AllOtherNonPaid',
                            'Affiliate_Att_AllOtherNonPaid']].rename(columns= {'CTV_Att_AllOtherNonPaid': 'CTV_attributed',
                            'Affiliate_Att_AllOtherNonPaid': 'All_Other_Paid_attributed' })

In [131]:
df_All_Other_Non_Paid['Channel'] = 'All Other Non Paid'

df_All_Other_Non_Paid['Affiliate_attributed'] = 0
#df_All_Other_Non_Paid['All_Other_Paid_attributed'] = 0
df_All_Other_Non_Paid['Display_attributed'] = 0
df_All_Other_Non_Paid['Email_attributed'] = 0
df_All_Other_Non_Paid['SearchBrand_attributed'] = 0
df_All_Other_Non_Paid['Search_NonBrand_attributed'] = 0
df_All_Other_Non_Paid['ShoppingAds_attributed'] = 0
df_All_Other_Non_Paid['Paid_Social_attributed'] = 0 
df_All_Other_Non_Paid['TV_attributed'] = 0 
#df_All_Other_Non_Paid['CTV_attributed'] = 0 

In [132]:
# Paid Search Attributed Orders:

df_All_Other_Non_Paid = df_All_Other_Non_Paid[['Date_W', 'Channel',
                               'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
                               'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
                               'Email_attributed', 'SearchBrand_attributed',
                               'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_All_Other_Non_Paid.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,All Other Non Paid,0,11.856476,0,0,9.143524,0,0,0,0,0
1,2019-04-08,All Other Non Paid,0,76.098288,0,0,103.901712,0,0,0,0,0
2,2019-04-15,All Other Non Paid,0,81.094500,0,0,57.905500,0,0,0,0,0
3,2019-04-22,All Other Non Paid,0,72.290056,0,0,43.709944,0,0,0,0,0
4,2019-04-29,All Other Non Paid,0,81.512520,0,0,34.487480,0,0,0,0,0


In [133]:
df_All_Other_Non_Paid.iloc[:,2:12].sum()

TV_attributed                    0.000000
CTV_attributed                3327.562124
Paid_Social_attributed           0.000000
Affiliate_attributed             0.000000
All_Other_Paid_attributed     1893.437876
Display_attributed               0.000000
Email_attributed                 0.000000
SearchBrand_attributed           0.000000
Search_NonBrand_attributed       0.000000
ShoppingAds_attributed           0.000000
dtype: float64

In [134]:
df_All_Other_Non_Paid.iloc[:,2:12].sum().sum()

5221.0

# TV Attribution

In [135]:
y = df1_Weekly['TV']
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     TV   R-squared (uncentered):                   0.905
Model:                            OLS   Adj. R-squared (uncentered):              0.894
Method:                 Least Squares   F-statistic:                              79.54
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    6.40e-13
Time:                        09:58:31   Log-Likelihood:                         -122.23
No. Observations:                  28   AIC:                                      250.5
Df Residuals:                      25   BIC:                                      254.4
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent        0.0001   3.59e-05      3.053      0.005    3.57e-05       0.000
CTV_Spent      -0.0004   9.91e-05     -4.106      0.000      -0.001      -0.000
nonTV_Spent     0.0001   3.27e-05      3.187      0.004    3.69e-05       0.000
==============================================================================
Omnibus:                        8.143   Durbin-Watson:                   1.230
Prob(Omnibus):                  0.017   Jarque-Bera (JB):                6.806
Skew:                           0.833   Prob(JB):                       0.0333
Kurtosis:                       4.749   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [136]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["TV"]
X1= df1_Weekly[['Affiliate_Spent', 
                'All_Other_Paid_Spent', 
                'Display_Spent',
               'Email_Spent',
               'Paid_Social_Spent',
                'Search_Brand_Spent',
               'Search_NonBrand_Spent',
               'Shopping_Ads_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                     TV   R-squared (uncentered):                   0.917
Model:                            OLS   Adj. R-squared (uncentered):              0.884
Method:                 Least Squares   F-statistic:                              27.60
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    3.53e-09
Time:                        09:58:32   Log-Likelihood:                         -120.37
No. Observations:                  28   AIC:                                      256.7
Df Residuals:                      20   BIC:                                      267.4
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent           0.0012      0.001      1.016      0.322      -0.001       0.004
All_Other_Paid_Spent   5.735e-05      0.000      0.132      0.896      -0.001       0.001
Display_Spent            -0.0016      0.002     -0.839      0.412      -0.005       0.002
Email_Spent              -0.0005      0.002     -0.358      0.724      -0.004       0.003
Paid_Social_Spent      9.494e-06   9.89e-05      0.096      0.924      -0.000       0.000
Search_Brand_Spent        0.0013      0.001      1.934      0.067      -0.000       0.003
Search_NonBrand_Spent     0.0038      0.001      2.992      0.007       0.001       0.006
Shopping_Ads_Spent       -0.0019      0.001     -1.775      0.091      -0.004       0.000
==============================================================================
Omnibus:                        5.010   Durbin-Watson:                   1.444
Prob(Omnibus):                  0.082   Jarque-Bera (JB):                4.644
Skew:                           0.190   Prob(JB):                       0.0981
Kurtosis:                       4.959   Cond. No.                         202.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
# All of TV goes to TV Attribution

In [138]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_TV = df1_Weekly[['Date_W','TV']].rename(columns= {'TV': 'TV_attributed'})
df_TV.head(3)

,Date_W,TV_attributed
0,2019-04-01,55
1,2019-04-08,198
2,2019-04-15,86


In [139]:
df_TV['Channel'] = 'TV'

df_TV['Affiliate_attributed'] = 0
df_TV['All_Other_Paid_attributed'] = 0
df_TV['Display_attributed'] = 0
df_TV['Email_attributed'] = 0
df_TV['SearchBrand_attributed'] = 0
df_TV['Search_NonBrand_attributed'] = 0
df_TV['ShoppingAds_attributed'] = 0
df_TV['Paid_Social_attributed'] = 0 
#df_TV['TV_attributed'] = 0 
df_TV['CTV_attributed'] = 0 

In [140]:
# Paid Search Attributed Orders:

df_TV = df_TV[['Date_W', 'Channel',
           'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
           'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
           'Email_attributed', 'SearchBrand_attributed',
           'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_TV.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,TV,55,0,0,0,0,0,0,0,0,0
1,2019-04-08,TV,198,0,0,0,0,0,0,0,0,0
2,2019-04-15,TV,86,0,0,0,0,0,0,0,0,0
3,2019-04-22,TV,75,0,0,0,0,0,0,0,0,0
4,2019-04-29,TV,59,0,0,0,0,0,0,0,0,0


In [141]:
df_TV.iloc[:,2:12].sum()

TV_attributed                 1483
CTV_attributed                   0
Paid_Social_attributed           0
Affiliate_attributed             0
All_Other_Paid_attributed        0
Display_attributed               0
Email_attributed                 0
SearchBrand_attributed           0
Search_NonBrand_attributed       0
ShoppingAds_attributed           0
dtype: int64

In [142]:
df_TV.iloc[:,2:12].sum().sum()

1483

# Display Attribution

In [143]:
y = df1_Weekly['Display']
X = df1_Weekly[['TV_Spent', 'CTV_Spent','nonTV_Spent']]

# Regression of y on all Channels:

result =sm.OLS(y,X).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                Display   R-squared (uncentered):                   0.861
Model:                            OLS   Adj. R-squared (uncentered):              0.844
Method:                 Least Squares   F-statistic:                              51.42
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    7.75e-11
Time:                        09:58:38   Log-Likelihood:                         -84.544
No. Observations:                  28   AIC:                                      175.1
Df Residuals:                      25   BIC:                                      179.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
TV_Spent     2.736e-05   9.36e-06      2.923      0.007    8.08e-06    4.66e-05
CTV_Spent   -4.548e-05   2.58e-05     -1.762      0.090   -9.86e-05    7.68e-06
nonTV_Spent  9.963e-06   8.51e-06      1.170      0.253   -7.57e-06    2.75e-05
==============================================================================
Omnibus:                        3.284   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.194   Jarque-Bera (JB):                2.099
Skew:                           0.108   Prob(JB):                        0.350
Kurtosis:                       4.324   Cond. No.                         22.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [144]:
# Get the Model Coefficients
TV_Coef = result.params[0]
CTV_Coef = result.params[1]
nonTV_Coef = result.params[2]
print(pd.DataFrame(result.params).rename(columns={0:'Coef'}))

                 Coef
TV_Spent     0.000027
CTV_Spent   -0.000045
nonTV_Spent  0.000010


In [145]:
# Predicting the Total Number of Paid_Search Orders using the regression result:
df1_Weekly['Display_Pred'] = ((df1_Weekly['TV_Spent']* TV_Coef) + (df1_Weekly['CTV_Spent']* CTV_Coef)+
                                  (df1_Weekly['nonTV_Spent']* nonTV_Coef))
df1_Weekly['Display_Pred'].sum()

312.5681378798008

In [146]:
y.sum()

306

In [147]:
# Attribute & Rescale:
df1_Weekly['TV_Att_Display'] = (df1_Weekly['TV_Spent']* TV_Coef)
#df1_Weekly['TV_Att_Display'] = (df1_Weekly['TV_Att_Display']/df1_Weekly['Display_Pred'])*y
df1_Weekly['TV_Att_Display'].sum()

331.431996048427

In [148]:
# Run the regression of residual orders on the other Spent Channels:

y= df1_Weekly["Display"]
X1= df1_Weekly[['Affiliate_Spent', 
                'All_Other_Paid_Spent', 
                'Display_Spent',
               'Email_Spent',
               'Paid_Social_Spent',
                'Search_Brand_Spent',
               'Search_NonBrand_Spent',
               'Shopping_Ads_Spent']]

# Regression of Paid_Search on all Channels:

result =sm.OLS(y,X1).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                Display   R-squared (uncentered):                   0.906
Model:                            OLS   Adj. R-squared (uncentered):              0.868
Method:                 Least Squares   F-statistic:                              24.10
Date:                Thu, 23 Jul 2020   Prob (F-statistic):                    1.18e-08
Time:                        09:58:41   Log-Likelihood:                         -79.018
No. Observations:                  28   AIC:                                      174.0
Df Residuals:                      20   BIC:                                      184.7
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Affiliate_Spent       -8.637e-05      0.000     -0.315      0.756      -0.001       0.000
All_Other_Paid_Spent   8.427e-05   9.93e-05      0.848      0.406      -0.000       0.000
Display_Spent             0.0002      0.000      0.417      0.681      -0.001       0.001
Email_Spent              -0.0004      0.000     -1.298      0.209      -0.001       0.000
Paid_Social_Spent      2.418e-07   2.26e-05      0.011      0.992   -4.69e-05    4.74e-05
Search_Brand_Spent        0.0006      0.000      3.760      0.001       0.000       0.001
Search_NonBrand_Spent     0.0006      0.000      1.958      0.064   -3.73e-05       0.001
Shopping_Ads_Spent       -0.0004      0.000     -1.634      0.118      -0.001       0.000
==============================================================================
Omnibus:                        3.334   Durbin-Watson:                   2.530
Prob(Omnibus):                  0.189   Jarque-Bera (JB):                1.910
Skew:                          -0.579   Prob(JB):                        0.385
Kurtosis:                       3.545   Cond. No.                         202.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [149]:
# Based on the Display findings, the number of orders are too small and the coeffients the model is giving are not significant
for attibuting the orders to spent channels. As a result, we will attribute all Display orders to Display.


SyntaxError: invalid syntax (<ipython-input-149-1c0226f20998>, line 2)

In [150]:
# Create a New Data Frame to Store Attributed Paid Search Orders:

df_Display = df1_Weekly[['Date_W','Display']].rename(columns= {'Display': 'Display_attributed'})
df_Display.head(3)

,Date_W,Display_attributed
0,2019-04-01,2
1,2019-04-08,28
2,2019-04-15,20


In [151]:
df_Display['Channel'] = 'Display'

df_Display['Affiliate_attributed'] = 0
df_Display['All_Other_Paid_attributed'] = 0
#df_Display['Display_attributed'] = 0
df_Display['Email_attributed'] = 0
df_Display['SearchBrand_attributed'] = 0
df_Display['Search_NonBrand_attributed'] = 0
df_Display['ShoppingAds_attributed'] = 0
df_Display['Paid_Social_attributed'] = 0 
df_Display['TV_attributed'] = 0 
df_Display['CTV_attributed'] = 0 

In [152]:
# Paid Search Attributed Orders:

df_Display = df_Display[['Date_W', 'Channel',
           'TV_attributed', 'CTV_attributed', 'Paid_Social_attributed', 
           'Affiliate_attributed', 'All_Other_Paid_attributed', 'Display_attributed',
           'Email_attributed', 'SearchBrand_attributed',
           'Search_NonBrand_attributed', 'ShoppingAds_attributed']]
df_Display.head()

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Display,0,0,0,0,0,2,0,0,0,0
1,2019-04-08,Display,0,0,0,0,0,28,0,0,0,0
2,2019-04-15,Display,0,0,0,0,0,20,0,0,0,0
3,2019-04-22,Display,0,0,0,0,0,32,0,0,0,0
4,2019-04-29,Display,0,0,0,0,0,16,0,0,0,0


In [153]:
df_Display.iloc[:,2:12].sum()

TV_attributed                   0
CTV_attributed                  0
Paid_Social_attributed          0
Affiliate_attributed            0
All_Other_Paid_attributed       0
Display_attributed            306
Email_attributed                0
SearchBrand_attributed          0
Search_NonBrand_attributed      0
ShoppingAds_attributed          0
dtype: int64

In [154]:
df_Display.iloc[:,2:12].sum().sum()

306

In [155]:
# Select between df_Paid_Serach or df_PaidOrganic_Search:

CHANNELS = [df_PaidOrganic_Search , df_Direct_Search , df_PaidOrganic_Social , df_ShoppingAds , df_Email, df_Affiliate ,
             df_All_Other_Non_Paid , df_TV, df_Display]
M3Solver = pd.concat(CHANNELS)
M3Solver

,Date_W,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,2019-04-01,Paid_Organic_Search,632.651954,154.403518,196.834123,38.110405,0.0,0.0,0.0,0.0,0,0
1,2019-04-08,Paid_Organic_Search,4161.802563,613.784264,1317.003156,316.410017,0.0,0.0,0.0,0.0,0,0
2,2019-04-15,Paid_Organic_Search,3778.494482,1126.573444,1307.576184,245.355889,0.0,0.0,0.0,0.0,0,0
3,2019-04-22,Paid_Organic_Search,3364.830229,978.485718,968.890476,213.793577,0.0,0.0,0.0,0.0,0,0
4,2019-04-29,Paid_Organic_Search,3276.926758,1199.528521,818.437545,223.107175,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
23,2019-09-09,Display,0.000000,0.000000,0.000000,0.000000,0.0,6.0,0.0,0.0,0,0
24,2019-09-16,Display,0.000000,0.000000,0.000000,0.000000,0.0,5.0,0.0,0.0,0,0
25,2019-09-23,Display,0.000000,0.000000,0.000000,0.000000,0.0,6.0,0.0,0.0,0,0
26,2019-09-30,Display,0.000000,0.000000,0.000000,0.000000,0.0,11.0,0.0,0.0,0,0


In [162]:
M3Solver_Orders = M3Solver.groupby('Channel').sum().reset_index()
M3Solver_Orders

,Channel,TV_attributed,CTV_attributed,Paid_Social_attributed,Affiliate_attributed,All_Other_Paid_attributed,Display_attributed,Email_attributed,SearchBrand_attributed,Search_NonBrand_attributed,ShoppingAds_attributed
0,Affiliate,0.000000,9988.257369,0.000000,147.619611,0.000000,0.000,0.000000,3891.123019,0,0
1,All Other Non Paid,0.000000,3327.562124,0.000000,0.000000,1893.437876,0.000,0.000000,0.000000,0,0
2,Direct,18135.393423,-699.401204,12316.084110,9108.235849,0.000000,0.000,5458.687823,0.000000,0,0
3,Display,0.000000,0.000000,0.000000,0.000000,0.000000,306.000,0.000000,0.000000,0,0
4,Email,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,14387.000000,0.000000,0,0
5,Paid_Organic_Search,71486.710825,24645.815244,29507.961989,7219.040504,0.000000,2791.281,0.000000,4696.190438,0,0
6,Shopping_Ads,10268.214707,0.000000,4911.277387,4305.236930,0.000000,0.000,1516.270976,0.000000,0,0
7,TV,1483.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0,0


In [168]:
#Attributed Orders by Channel:
Attributed_Orders_By_Channel = pd.DataFrame(M3Solver.groupby('Channel').sum()).sum()
Attributed_Orders_By_Channel

TV_attributed                 101373.318956
CTV_attributed                 37262.233534
Paid_Social_attributed         46735.323485
Affiliate_attributed           20780.132895
All_Other_Paid_attributed       1893.437876
Display_attributed              3097.281000
Email_attributed               21361.958798
SearchBrand_attributed          8587.313457
Search_NonBrand_attributed         0.000000
ShoppingAds_attributed             0.000000
dtype: float64

In [165]:
# TV Attributed Orders:
pd.DataFrame(M3Solver.groupby('Channel').sum()).sum()[0]

101373.31895590175

In [166]:
# TV CPO: 
df1_Weekly['TV_Spent'].sum()/pd.DataFrame(M3Solver.groupby('Channel').sum()).sum()[0]

119.51558067532967

In [167]:
# CTV CPO:
df1_Weekly['CTV_Spent'].sum()/pd.DataFrame(M3Solver.groupby('Channel').sum()).sum()[1]

115.35475923940312

In [161]:
# Social CPO:
df1_Weekly['Paid_Social_Spent'].sum()/pd.DataFrame(M3Solver.groupby('Channel').sum()).sum()[2]

218.17309135061137

In [187]:
Spent_By_Channel = pd.DataFrame(round(df1_Weekly.iloc[:, 2:13].sum(),2)).rename(columns={0:'Spent'}).sort_values(ascending=False, by ='Spent')
Spent_By_Channel

,Spent
Total_Spent,30704082.20
TV_Spent,12115691.08
Paid_Social_Spent,10196390.00
CTV_Spent,4298375.98
Search_Brand_Spent,1147002.00
Shopping_Ads_Spent,861697.00
Affiliate_Spent,672946.00
Email_Spent,498781.00
Search_NonBrand_Spent,355165.00
All_Other_Paid_Spent,288486.76


In [185]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('M3SOlver_Results_07232020.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
M3Solver.to_excel(writer, sheet_name='Data')
M3Solver_Orders.to_excel(writer, sheet_name='Pivot')
Attributed_Orders_By_Channel.to_excel(writer, sheet_name='OrdersByChannel')
Spent_By_Channel.to_excel(writer, sheet_name='SpentByChannel')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [186]:
df1_Weekly.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 73 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Date_W                               28 non-null     datetime64[ns]
 1   Total_Orders                         28 non-null     int64         
 2   Total_Spent                          28 non-null     float64       
 3   TV_Spent                             28 non-null     float64       
 4   CTV_Spent                            28 non-null     float64       
 5   Affiliate_Spent                      28 non-null     float64       
 6   All_Other_Paid_Spent                 28 non-null     float64       
 7   Display_Spent                        28 non-null     float64       
 8   Email_Spent                          28 non-null     float64       
 9   Paid_Social_Spent                    28 non-null     float64       
 10  Search_Brand_Spe

In [196]:
df_Weekly.iloc[:, 13:30].corr()*100

,All_Other_Non-Paid,All_Other_Paid,Direct,Display,Email,Organic_Search,Organic_Social,Search_Brand_,Search_Non-Brand,Shopping_Ads,TV,PR,Video,Partnership_Marketing,Paid_Social,Paid_Search
All_Other_Non-Paid,100.000000,36.107685,-29.779543,-0.451526,-9.635721,6.358129,10.954004,38.325751,55.719289,-24.326334,-4.232883,-6.679048,-3.551158,-1.753690,-17.202420,-26.579013
All_Other_Paid,36.107685,100.000000,-19.470292,-22.481990,-45.922574,-4.654967,28.161976,72.130886,72.628675,-65.896535,-32.549546,-13.981575,-14.640675,-19.260758,-19.391703,-68.373294
Direct,-29.779543,-19.470292,100.000000,55.785860,54.838732,45.190190,24.382963,-31.992349,-35.275288,57.090618,59.748698,23.595073,21.676425,19.633790,-2.533192,56.359578
Display,-0.451526,-22.481990,55.785860,100.000000,66.446775,55.648295,37.891786,-24.170221,-23.140975,68.969523,78.430523,20.602613,24.638866,29.869224,-55.078386,66.009895
Email,-9.635721,-45.922574,54.838732,66.446775,100.000000,55.177496,16.734734,-48.840980,-45.747053,85.012781,72.228760,22.731598,32.923540,43.667581,4.015311,82.293382
Organic_Search,6.358129,-4.654967,45.190190,55.648295,55.177496,100.000000,73.848247,15.676704,4.878415,52.994419,73.947067,27.457109,23.181470,14.210328,10.121538,46.560250
Organic_Social,10.954004,28.161976,24.382963,37.891786,16.734734,73.848247,100.000000,55.808575,41.191648,6.718393,43.016247,16.983825,14.713714,-5.221323,-13.750793,3.662345
Search_Brand_,38.325751,72.130886,-31.992349,-24.170221,-48.840980,15.676704,55.808575,100.000000,89.110869,-71.540170,-32.787948,-15.179125,-15.894678,-20.910481,-18.373368,-74.228891
Search_Non-Brand,55.719289,72.628675,-35.275288,-23.140975,-45.747053,4.878415,41.191648,89.110869,100.000000,-68.080384,-29.710811,-14.444932,-15.125875,-19.899071,-18.944479,-70.639224
Shopping_Ads,-24.326334,-65.896535,57.090618,68.969523,85.012781,52.994419,6.718393,-71.540170,-68.080384,100.000000,80.617901,27.860721,33.436696,31.906865,9.679495,97.069312


In [190]:
df1_Weekly.iloc[:, 13:30].corr()*100

,Affiliate,All_Other_Non-Paid,All_Other_Paid,Direct,Display,Email,Organic_Search,Organic_Social,Search_Brand_,Search_Non-Brand,Shopping_Ads,TV,PR,Video,Partnership_Marketing,Paid_Social,Paid_Search
Affiliate,100.000000,48.889939,NaN,0.644903,7.507118,-31.278430,20.667385,-8.294982,NaN,NaN,18.627234,1.445423,11.338779,11.338779,-15.042061,-1.391044,16.936984
All_Other_Non-Paid,48.889939,100.000000,NaN,3.837210,-22.985550,16.056949,-3.791183,-18.432817,NaN,NaN,10.923358,-18.168444,8.103618,8.103618,15.426200,-11.157606,13.740762
All_Other_Paid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Direct,0.644903,3.837210,NaN,100.000000,71.052134,77.514532,87.767914,83.116550,NaN,NaN,94.510703,66.851812,39.460752,39.460752,19.371807,93.656542,89.869443
Display,7.507118,-22.985550,NaN,71.052134,100.000000,36.151469,68.415376,59.080072,NaN,NaN,72.938210,64.595204,16.847181,16.847181,10.320652,78.338250,72.513768
Email,-31.278430,16.056949,NaN,77.514532,36.151469,100.000000,50.775026,65.571043,NaN,NaN,63.268559,39.523409,39.389336,39.389336,34.714055,68.118911,65.725027
Organic_Search,20.667385,-3.791183,NaN,87.767914,68.415376,50.775026,100.000000,81.567159,NaN,NaN,92.806897,74.401937,51.123404,51.123404,-1.551394,87.503853,73.905998
Organic_Social,-8.294982,-18.432817,NaN,83.116550,59.080072,65.571043,81.567159,100.000000,NaN,NaN,78.290461,61.605461,40.068637,40.068637,2.797150,75.139709,76.127109
Search_Brand_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Search_Non-Brand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [193]:
df.iloc[:,1:18].corr()*100

,Affiliate,All_Other_Non-Paid,All_Other_Paid,Direct,Display,Email,Organic_Search,Organic_Social,Paid_Search,Paid_Social,Partnership_Marketing,PR,Search_Brand_,Search_Non-Brand,Shopping_Ads,TV,Video
Affiliate,100.000000,1.023100,15.788391,-5.906195,25.114141,3.294082,53.174535,48.378611,0.539792,-12.741217,0.027728,1.910388,45.817168,26.548719,0.687814,19.225496,0.936103
All_Other_Non-Paid,1.023100,100.000000,29.825333,-29.694054,0.721489,-7.737389,2.310761,8.752348,-25.783813,-17.702568,-0.164868,-0.682510,34.993419,49.302950,-22.625746,-2.501187,-2.082041
All_Other_Paid,15.788391,29.825333,100.000000,-28.391258,-19.337535,-36.747698,-1.908057,23.809827,-59.698407,-14.537487,-7.722824,-4.570773,63.264883,61.630958,-56.901135,-25.452331,-5.419780
Direct,-5.906195,-29.694054,-28.391258,100.000000,48.335150,43.558357,31.158207,13.621711,45.681707,-9.761308,8.004179,7.504185,-29.795068,-31.561703,45.996769,43.947320,6.183714
Display,25.114141,0.721489,-19.337535,48.335150,100.000000,60.721405,52.651461,33.421951,64.352374,-52.980558,13.493336,6.559544,-23.436318,-21.862426,66.101115,68.553496,11.626300
Email,3.294082,-7.737389,-36.747698,43.558357,60.721405,100.000000,47.181196,12.729610,74.820777,0.636350,20.818026,9.055327,-44.913019,-41.259043,76.748243,57.358509,14.903900
Organic_Search,53.174535,2.310761,-1.908057,31.158207,52.651461,47.181196,100.000000,60.826564,37.611134,5.566946,4.003151,3.681068,14.500315,3.854366,48.365093,59.768036,4.252565
Organic_Social,48.378611,8.752348,23.809827,13.621711,33.421951,12.729610,60.826564,100.000000,2.191206,-11.452906,-2.274595,4.351361,49.017555,34.648987,5.236094,31.810049,6.111786
Paid_Search,0.539792,-25.783813,-59.698407,45.681707,64.352374,74.820777,37.611134,2.191206,100.000000,8.866350,15.056006,10.262989,-73.057925,-67.708051,92.629324,65.980730,11.457541
Paid_Social,-12.741217,-17.702568,-14.537487,-9.761308,-52.980558,0.636350,5.566946,-11.452906,8.866350,100.000000,-2.586576,4.220458,-17.757987,-17.989322,6.854873,-7.446623,2.856763


In [195]:
df1.iloc[:,1:18].corr()*100

,Affiliate,All_Other_Non-Paid,All_Other_Paid,Direct,Display,Email,Organic_Search,Organic_Social,Paid_Search,Paid_Social,Partnership_Marketing,PR,Search_Brand_,Search_Non-Brand,Shopping_Ads,TV,Video
Affiliate,100.000000,17.311252,NaN,-12.529963,-8.431586,-29.724797,7.538136,-12.696966,-9.825952,-14.507631,-12.674508,6.188653,NaN,NaN,-1.059677,17.297149,0.162460
All_Other_Non-Paid,17.311252,100.000000,NaN,-13.855337,-13.089027,2.341558,-21.806413,-13.219279,-13.372062,-23.627369,4.217166,20.945605,NaN,NaN,-11.201367,-8.064411,-4.796836
All_Other_Paid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Direct,-12.529963,-13.855337,NaN,100.000000,44.434396,65.639426,74.133399,60.590227,66.810479,80.721960,1.460091,12.486577,NaN,NaN,78.727137,56.545656,2.656088
Display,-8.431586,-13.089027,NaN,44.434396,100.000000,24.814490,34.426988,26.320802,46.320023,52.980886,-0.606191,15.983063,NaN,NaN,42.697571,33.620127,2.557290
Email,-29.724797,2.341558,NaN,65.639426,24.814490,100.000000,35.849691,42.966304,50.358770,54.427651,9.378096,18.148807,NaN,NaN,49.706428,31.468886,18.148807
Organic_Search,7.538136,-21.806413,NaN,74.133399,34.426988,35.849691,100.000000,52.602009,30.172206,67.897384,-2.693100,3.297465,NaN,NaN,77.261031,52.651252,4.659250
Organic_Social,-12.696966,-13.219279,NaN,60.590227,26.320802,42.966304,52.602009,100.000000,43.024005,50.177621,-1.632682,11.315463,NaN,NaN,52.583562,39.611876,10.208515
Paid_Search,-9.825952,-13.372062,NaN,66.810479,46.320023,50.358770,30.172206,43.024005,100.000000,64.778295,4.141612,18.785434,NaN,NaN,60.887864,44.336944,12.576168
Paid_Social,-14.507631,-23.627369,NaN,80.721960,52.980886,54.427651,67.897384,50.177621,64.778295,100.000000,6.151922,15.770685,NaN,NaN,77.343799,52.944703,15.578601
